In [12]:
import requests
from chembl_webresource_client.new_client import new_client
import pandas as pd
import numpy as np
from typing import List, Dict, Union, Optional
from functools import lru_cache
import time
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
import os
from pathlib import Path

# Настройка сессии с повторными попытками
session = requests.Session()
retries = Retry(
    total=5,
    backoff_factor=1,
    status_forcelist=[500, 502, 503, 504]
)
session.mount('https://', HTTPAdapter(max_retries=retries))

# Установка таймаута по умолчанию для ChEMBL клиента
new_client.timeout = 30

def uniprot_to_chembl(uniprot_id: str) -> Optional[str]:
    """Получает ChEMBL ID из UniProt ID."""
    target = new_client.target
    try:
        res = target.filter(target_components__accession=uniprot_id)
        res_df = pd.DataFrame(res)
        
        if res_df.empty:
            return None
        return res_df['target_chembl_id'].iloc[0]
    except Exception as e:
        print(f"Ошибка при получении ChEMBL ID для UniProt {uniprot_id}: {e}")
        return None

def search_gene_in_uniprot_filtered(gene_name: str, organism: str = "Homo sapiens") -> Union[str, List[Dict]]:
    """Ищет информацию о гене в UniProt."""
    url = "https://rest.uniprot.org/uniprotkb/search"
    query = f'gene_exact:"{gene_name}" AND reviewed:true AND taxonomy_name:"{organism}"'
    
    params = {
        "query": query,
        "format": "json",
        "fields": "accession,gene_names,protein_name",
        "size": 10
    }
    
    try:
        response = session.get(url, params=params, timeout=30)
        response.raise_for_status()
        
        results = response.json().get("results", [])
        if not results:
            return f"No results found for gene '{gene_name}'"
        
        gene_info = [
            {
                "UniProt_ID": result["primaryAccession"],
                "Gene_Names": result["genes"][0]["geneName"]["value"] if result.get("genes") else "N/A",
                "Protein_Name": result["proteinDescription"]["recommendedName"]["fullName"]["value"],
            }
            for result in results
        ]
        
        for gene in gene_info:
            uniprot_id = gene["UniProt_ID"]
            chembl_id = uniprot_to_chembl(uniprot_id)
            gene["ChEMBL_ID"] = chembl_id if chembl_id else f"No ChEMBL ID found for {uniprot_id}"
            time.sleep(0.1)  # Задержка между запросами
        
        return gene_info
    except Exception as e:
        print(f"Ошибка при поиске гена {gene_name}: {e}")
        return f"Error searching gene {gene_name}: {str(e)}"

@lru_cache(maxsize=1000)
def get_parent_info(chembl_id: str) -> dict:
    """Получает информацию о родительском соединении для данного ChEMBL ID."""
    molecule = new_client.molecule
    molecule_form = new_client.molecule_form
    
    try:
        # Проверяем, есть ли родительское соединение
        parent_data = list(molecule_form.filter(molecule_chembl_id=chembl_id).only('parent_chembl_id'))
        
        if parent_data and 'parent_chembl_id' in parent_data[0]:
            parent_id = parent_data[0]['parent_chembl_id']
        else:
            # Если нет родителя, считаем текущий ID родительским
            parent_id = chembl_id
        
        # Получаем информацию о родительском соединении
        mol_data = list(molecule.filter(molecule_chembl_id=parent_id).only('pref_name', 'molecule_chembl_id'))
        pref_name = mol_data[0].get('pref_name', 'Unknown') if mol_data else 'Unknown'
        
        return {
            'parent_chembl_id': parent_id,
            'parent_pref_name': pref_name,
            'is_parent': parent_id == chembl_id
        }
    except Exception as e:
        print(f"Error getting parent info for {chembl_id}: {e}")
        return {
            'parent_chembl_id': chembl_id,
            'parent_pref_name': 'Unknown',
            'is_parent': True
        }

def filter_parent_drugs(drugs_list: List[Dict]) -> List[Dict]:
    """Фильтрует список препаратов, оставляя по одному представителю из каждого семейства.
       Сохраняет 'Yes' в Drug Mechanism при замене дочернего с 'Yes' на родительский."""
    parent_groups = {}
    
    # 1. Группируем все препараты по семействам
    for drug in drugs_list:
        try:
            if not drug.get('Drug ChEMBL ID'):
                continue
                
            parent_info = get_parent_info(drug['Drug ChEMBL ID'])
            parent_id = parent_info['parent_chembl_id']
            
            if parent_id not in parent_groups:
                parent_groups[parent_id] = {
                    'parent_info': parent_info,
                    'drugs': [],
                    'has_mechanism': False
                }
            
            parent_groups[parent_id]['drugs'].append(drug)
            if drug.get('Drug Mechanism ChEMBL') == 'Yes':
                parent_groups[parent_id]['has_mechanism'] = True
                
        except Exception as e:
            print(f"Ошибка при группировке препаратов: {e}")
            continue
    
    filtered_drugs = []
    
    # 2. Обрабатываем каждое семейство
    for parent_id, group in parent_groups.items():
        drugs_in_group = group['drugs']
        has_mechanism = group['has_mechanism']
        
        # Если в группе только один препарат - оставляем как есть
        if len(drugs_in_group) == 1:
            filtered_drugs.append(drugs_in_group[0])
            continue
        
        # Находим родительский препарат в группе
        parent_drug = next(
            (d for d in drugs_in_group if d['Drug ChEMBL ID'] == parent_id),
            None
        )
        
        # Если есть родительский препарат
        if parent_drug:
            # Если в семействе есть препараты с механизмом, сохраняем 'Yes' для родителя
            if has_mechanism:
                parent_drug['Drug Mechanism ChEMBL'] = 'Yes'
            filtered_drugs.append(parent_drug)
        else:
            # Если нет родительского - выбираем препарат с максимальным pChEMBL
            best_drug = max(
                drugs_in_group,
                key=lambda x: float(x['pChEMBL value']) if x.get('pChEMBL value', 'N/A') != 'N/A' else -1
            )
            filtered_drugs.append(best_drug)
    
    return filtered_drugs

def get_action_type(chembl_id: str) -> str:
    try:
        client = new_client
        mechanisms = client.mechanism.filter(molecule_chembl_id=chembl_id)
        
        action_types = set()
        for mechanism in mechanisms:
            action_type = mechanism.get("action_type")
            if action_type:
                action_types.add(action_type)
        
        return "; ".join(action_types) if action_types else "N/A"
    except Exception as e:
        print(f"Ошибка при получении типа действия для {chembl_id}: {e}")
        return "N/A"

def get_mechanism_details(chembl_id: str) -> List[Dict]:
    try:
        client = new_client
        mechanisms = client.mechanism.filter(molecule_chembl_id=chembl_id)
        
        details = []
        for mechanism in mechanisms:
            mechanism_of_action = mechanism.get("mechanism_of_action", "N/A")
            
            details.append({
                "Mechanism of Action ChEMBL": mechanism_of_action,  # Изменено здесь
            })
        
        return details if details else [{"Mechanism of Action ChEMBL": "N/A"}]  # И здесь
    except Exception as e:
        print(f"Ошибка при получении механизма действия для {chembl_id}: {e}")
        return [{"Mechanism of Action ChEMBL": "N/A"}]  # И здесь

def get_max_phase(chembl_id: str) -> str:
    try:
        client = new_client
        molecule = client.molecule.get(chembl_id)
        max_phase = molecule.get("max_phase", None)
        
        if max_phase is None:
            return ""
        
        if max_phase == "0.0":
            return "Preclinical"
        elif max_phase == "1.0":
            return "Phase 1"
        elif max_phase == "2.0":
            return "Phase 2"
        elif max_phase == "3.0":
            return "Phase 3"
        elif max_phase == "4.0":
            return "Approved"
        else:
            return f"Unknown phase ({max_phase})"
    except Exception as e:
        print(f"Ошибка при получении фазы для {chembl_id}: {e}")
        return "N/A"

def get_pchembl_value(molecule_chembl_id: str, target_chembl_id: str) -> str:
    try:
        activities = new_client.activity.filter(
            molecule_chembl_id=molecule_chembl_id,
            target_chembl_id=target_chembl_id,
            pchembl_value__isnull=False
        ).only("pchembl_value")
        return activities[0]["pchembl_value"] if activities else "N/A"
    except Exception as e:
        print(f"Error fetching pChEMBL value for {molecule_chembl_id}: {e}")
        return "N/A"

def get_activity_data(molecule_chembl_id: str, target_chembl_id: str) -> Dict:
    """Возвращает объединенные данные активности: значение + единицы измерения"""
    try:
        activities = new_client.activity.filter(
            molecule_chembl_id=molecule_chembl_id,
            target_chembl_id=target_chembl_id,
            standard_type__in=["IC50", "Kd", "Ki", "EC50"]
        ).only("standard_value", "standard_type", "standard_units")
        
        if not activities:
            return {
                "Activity value ChEMBL": "N/A",
                "Activity type ChEMBL": "N/A"
            }
        
        activity = activities[0]
        value = activity.get('standard_value')
        units = activity.get('standard_units')
        act_type = activity.get('standard_type')
        
        if act_type == "Activity":
            act_type = "N/A"
        
        if value and units and value != "N/A" and units != "N/A":
            activity_value = f"= {value} {units}"
        else:
            activity_value = "N/A"
        
        return {
            "Activity value ChEMBL": activity_value,
            "Activity type ChEMBL": act_type
        }
    except Exception as e:
        print(f"Ошибка при получении данных активности для {molecule_chembl_id}: {e}")
        return {
            "Activity value ChEMBL": "N/A",
            "Activity type ChEMBL": "N/A"
        }

def get_drugs_for_target(chembl_id: str) -> List[Dict]:
    try:
        client = new_client
        
        mechanism_drugs = []
        mechanisms = client.mechanism.filter(target_chembl_id=chembl_id)
        for mech in mechanisms:
            try:
                drug_id = mech['molecule_chembl_id']
                drug = client.molecule.get(drug_id)
                
                activity_data = get_activity_data(drug_id, chembl_id)
                mechanism_details = get_mechanism_details(drug_id)
                
                if not mechanism_details:
                    print(f"Нет данных о механизме для {drug_id}")
                    continue
                    
                mechanism_drugs.append({
                    "Drug ChEMBL ID": drug_id,
                    "Drug name": drug.get('pref_name', 'N/A'),
                    "Drug Mechanism ChEMBL": "Yes",
                    "Activity value ChEMBL": activity_data["Activity value ChEMBL"],
                    "Activity type ChEMBL": activity_data["Activity type ChEMBL"],
                    "pChEMBL value": get_pchembl_value(drug_id, chembl_id),
                    "Max phase ChEMBL": get_max_phase(drug_id),
                    "Action type ChEMBL": get_action_type(drug_id),
                    "Mechanism of Action ChEMBL": mechanism_details[0]["Mechanism of Action ChEMBL"]
                })
                time.sleep(0.1)
            except Exception as e:
                print(f"Ошибка при обработке механизма {mech.get('molecule_chembl_id')}: {str(e)}")
                continue
        
        mechanism_ids = {drug["Drug ChEMBL ID"] for drug in mechanism_drugs}
        activity_drugs = []
        
        activities = client.activity.filter(
            target_chembl_id=chembl_id,
            pchembl_value__isnull=False,
            standard_type__in=["IC50", "Kd", "Ki", "EC50"]
        ).order_by('-pchembl_value')[:100]
        
        for activity in activities:
            try:
                if activity['molecule_chembl_id'] not in mechanism_ids:
                    value = activity.get('standard_value', 'N/A')
                    units = activity.get('standard_units', 'N/A')
                    act_type = activity.get('standard_type', 'N/A')
                    
                    if act_type == "Activity":
                        act_type = "N/A"
                    
                    activity_value = f"= {value} {units}" if value != "N/A" and units != "N/A" else "N/A"
                    
                    activity_drugs.append({
                        "Drug ChEMBL ID": activity['molecule_chembl_id'],
                        "Drug name": activity['molecule_pref_name'],
                        "Drug Mechanism ChEMBL": "No",
                        "Activity value ChEMBL": activity_value,  # Исправлено
                        "Activity type ChEMBL": act_type,          # Исправлено
                        "pChEMBL value": activity['pchembl_value'],
                        "Max phase ChEMBL": get_max_phase(activity['molecule_chembl_id']),  # Исправлено
                        "Action type ChEMBL": get_action_type(activity['molecule_chembl_id']),  # Исправлено
                        "Mechanism of Action ChEMBL": "N/A"
                    })
                    time.sleep(0.1)
            except Exception as e:
                print(f"Ошибка при обработке активности {activity}: {e}")
                continue
        
        all_drugs = mechanism_drugs + activity_drugs
        return filter_parent_drugs(all_drugs)
    except Exception as e:
        print(f"Ошибка при получении препаратов для цели {chembl_id}: {e}")
        return []

def get_gene_drug_info(gene_names: List[str]) -> List[Dict]:
    result = []
    
    for gene_name in gene_names:
        try:
            print(f"Обрабатывается ген: {gene_name}")
            gene_info_list = search_gene_in_uniprot_filtered(gene_name)
            
            if isinstance(gene_info_list, str):
                result.append({"Gene": gene_name, "Error": gene_info_list})
                continue
            
            unique_drugs_for_gene = {}

            for gene_info in gene_info_list:
                try:
                    if isinstance(gene_info['ChEMBL_ID'], str) and gene_info['ChEMBL_ID'].startswith("No ChEMBL"):
                        continue
                        
                    drugs = get_drugs_for_target(gene_info['ChEMBL_ID'])  # Только 1 аргумент!
                    
                    for drug in drugs:
                        chembl_id = drug["Drug ChEMBL ID"]
                        if chembl_id not in unique_drugs_for_gene or unique_drugs_for_gene[chembl_id]["pChEMBL value"] < drug["pChEMBL value"]:
                            unique_drugs_for_gene[chembl_id] = drug
                except Exception as e:
                    print(f"Ошибка при обработке информации о гене {gene_name}: {e}")
                    continue
            
            if not gene_info_list or 'UniProt_ID' not in gene_info_list[0]:
                continue
                
            for drug in unique_drugs_for_gene.values():
                result.append({
                    "Gene": gene_name,
                    "UniProt ID": gene_info_list[0]['UniProt_ID'],
                    "Protein name": gene_info_list[0]['Protein_Name'],
                    "Protein ChEMBL ID": gene_info_list[0]['ChEMBL_ID'],
                    **drug
                })
                
        except Exception as e:
            print(f"Ошибка при обработке гена {gene_name}: {e}")
            result.append({"Gene": gene_name, "Error": str(e)})
            continue

    return result

if __name__ == "__main__":
    # Замените этот путь на ваш реальный путь к файлу
    input_file = r"C:\Users\rusla\OneDrive\Рабочий стол\Диплом\Гены\thresholds\gene_selection\ALL\ALL_depscore_-0.2_zscore_-0.5.csv"
    
    # Чтение генов из файла
    try:
        genes_df = pd.read_csv(input_file)
        gene_names = genes_df['Gene'].tolist()
        print(f"Загружено {len(gene_names)} генов из файла")
    except Exception as e:
        print(f"Ошибка при чтении файла {input_file}: {e}")
        raise
    
    # Получаем имя выходного файла (без пути)
    output_filename = os.path.basename(input_file)
    
    start_time = time.time()
    print("\nНачинаем обработку с динамической фильтрацией pChEMBL")
    
    try:
        gene_drug_info = get_gene_drug_info(gene_names)
        
        # Определяем полный список всех возможных колонок
        all_columns = [
            "Gene", "UniProt ID", "Protein name", "Protein ChEMBL ID",
            "Drug ChEMBL ID", "Drug name", "Drug Mechanism ChEMBL",
            "Activity type ChEMBL", "Activity value ChEMBL",
            "pChEMBL value", "Max phase ChEMBL", "Action type ChEMBL", 
            "Mechanism of Action ChEMBL"
        ]
        
        # Создаем DataFrame из результатов
        df = pd.DataFrame(gene_drug_info)
        
        # Добавляем отсутствующие колонки и заполняем их NA
        for col in all_columns:
            if col not in df.columns:
                df[col] = pd.NA
        
        # Упорядочиваем колонки согласно нашему списку
        df = df[all_columns]
        
        # Фильтруем ошибочные записи (где нет Drug ChEMBL ID)
        df = df[df['Drug ChEMBL ID'].notna()]
        
        # Сохраняем в текущую директорию с тем же именем файла
        df.to_csv(output_filename, index=False, encoding='utf-8')
        
        elapsed_time = time.time() - start_time
        
        # Собираем статистику
        num_drugs = len(df)
        num_mechanisms = len(df[df['Drug Mechanism ChEMBL'] == 'Yes'])
        
        print(f"Файл {output_filename} сохранён. Время выполнения: {elapsed_time:.2f} секунд")
        print(f"Статистика: {num_drugs} препаратов, из них {num_mechanisms} с известным механизмом")
        
    except Exception as e:
        print(f"Критическая ошибка при обработке: {e}")

Загружено 907 генов из файла

Начинаем обработку с динамической фильтрацией pChEMBL
Обрабатывается ген: ABHD11
Обрабатывается ген: ACAP3
Обрабатывается ген: ACIN1
Обрабатывается ген: ACLY
Обрабатывается ген: ACO2
Обрабатывается ген: ACSL4
Обрабатывается ген: ACTL6A
Обрабатывается ген: ACTR10
Обрабатывается ген: ADGRB2
Обрабатывается ген: ADSL
Обрабатывается ген: ADSS2
Обрабатывается ген: AES
Обрабатывается ген: AGPS
Обрабатывается ген: AHCY
Обрабатывается ген: AHDC1
Обрабатывается ген: AJM1
Обрабатывается ген: AK2
Обрабатывается ген: AKAP17A
Обрабатывается ген: ALAD
Обрабатывается ген: ALAS1
Обрабатывается ген: ALYREF
Обрабатывается ген: ANAPC7
Обрабатывается ген: ANKH
Обрабатывается ген: ANKRD36
Обрабатывается ген: ANKRD62P1-PARP4P3
Обрабатывается ген: ANKUB1
Обрабатывается ген: AP1M1
Обрабатывается ген: AP2M1
Обрабатывается ген: AP2S1
Обрабатывается ген: APC
Обрабатывается ген: APOBEC3F
Обрабатывается ген: AQR
Обрабатывается ген: ARHGAP19-SLIT1
Обрабатывается ген: ARHGAP45
Обрабатыва

In [13]:
import requests
from chembl_webresource_client.new_client import new_client
import pandas as pd
import numpy as np
from typing import List, Dict, Union, Optional
from functools import lru_cache
import time
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
import os
from pathlib import Path

# Настройка сессии с повторными попытками
session = requests.Session()
retries = Retry(
    total=5,
    backoff_factor=1,
    status_forcelist=[500, 502, 503, 504]
)
session.mount('https://', HTTPAdapter(max_retries=retries))

# Установка таймаута по умолчанию для ChEMBL клиента
new_client.timeout = 30

def uniprot_to_chembl(uniprot_id: str) -> Optional[str]:
    """Получает ChEMBL ID из UniProt ID."""
    target = new_client.target
    try:
        res = target.filter(target_components__accession=uniprot_id)
        res_df = pd.DataFrame(res)
        
        if res_df.empty:
            return None
        return res_df['target_chembl_id'].iloc[0]
    except Exception as e:
        print(f"Ошибка при получении ChEMBL ID для UniProt {uniprot_id}: {e}")
        return None

def search_gene_in_uniprot_filtered(gene_name: str, organism: str = "Homo sapiens") -> Union[str, List[Dict]]:
    """Ищет информацию о гене в UniProt."""
    url = "https://rest.uniprot.org/uniprotkb/search"
    query = f'gene_exact:"{gene_name}" AND reviewed:true AND taxonomy_name:"{organism}"'
    
    params = {
        "query": query,
        "format": "json",
        "fields": "accession,gene_names,protein_name",
        "size": 10
    }
    
    try:
        response = session.get(url, params=params, timeout=30)
        response.raise_for_status()
        
        results = response.json().get("results", [])
        if not results:
            return f"No results found for gene '{gene_name}'"
        
        gene_info = [
            {
                "UniProt_ID": result["primaryAccession"],
                "Gene_Names": result["genes"][0]["geneName"]["value"] if result.get("genes") else "N/A",
                "Protein_Name": result["proteinDescription"]["recommendedName"]["fullName"]["value"],
            }
            for result in results
        ]
        
        for gene in gene_info:
            uniprot_id = gene["UniProt_ID"]
            chembl_id = uniprot_to_chembl(uniprot_id)
            gene["ChEMBL_ID"] = chembl_id if chembl_id else f"No ChEMBL ID found for {uniprot_id}"
            time.sleep(0.1)  # Задержка между запросами
        
        return gene_info
    except Exception as e:
        print(f"Ошибка при поиске гена {gene_name}: {e}")
        return f"Error searching gene {gene_name}: {str(e)}"

@lru_cache(maxsize=1000)
def get_parent_info(chembl_id: str) -> dict:
    """Получает информацию о родительском соединении для данного ChEMBL ID."""
    molecule = new_client.molecule
    molecule_form = new_client.molecule_form
    
    try:
        # Проверяем, есть ли родительское соединение
        parent_data = list(molecule_form.filter(molecule_chembl_id=chembl_id).only('parent_chembl_id'))
        
        if parent_data and 'parent_chembl_id' in parent_data[0]:
            parent_id = parent_data[0]['parent_chembl_id']
        else:
            # Если нет родителя, считаем текущий ID родительским
            parent_id = chembl_id
        
        # Получаем информацию о родительском соединении
        mol_data = list(molecule.filter(molecule_chembl_id=parent_id).only('pref_name', 'molecule_chembl_id'))
        pref_name = mol_data[0].get('pref_name', 'Unknown') if mol_data else 'Unknown'
        
        return {
            'parent_chembl_id': parent_id,
            'parent_pref_name': pref_name,
            'is_parent': parent_id == chembl_id
        }
    except Exception as e:
        print(f"Error getting parent info for {chembl_id}: {e}")
        return {
            'parent_chembl_id': chembl_id,
            'parent_pref_name': 'Unknown',
            'is_parent': True
        }

def filter_parent_drugs(drugs_list: List[Dict]) -> List[Dict]:
    """Фильтрует список препаратов, оставляя по одному представителю из каждого семейства.
       Сохраняет 'Yes' в Drug Mechanism при замене дочернего с 'Yes' на родительский."""
    parent_groups = {}
    
    # 1. Группируем все препараты по семействам
    for drug in drugs_list:
        try:
            if not drug.get('Drug ChEMBL ID'):
                continue
                
            parent_info = get_parent_info(drug['Drug ChEMBL ID'])
            parent_id = parent_info['parent_chembl_id']
            
            if parent_id not in parent_groups:
                parent_groups[parent_id] = {
                    'parent_info': parent_info,
                    'drugs': [],
                    'has_mechanism': False
                }
            
            parent_groups[parent_id]['drugs'].append(drug)
            if drug.get('Drug Mechanism ChEMBL') == 'Yes':
                parent_groups[parent_id]['has_mechanism'] = True
                
        except Exception as e:
            print(f"Ошибка при группировке препаратов: {e}")
            continue
    
    filtered_drugs = []
    
    # 2. Обрабатываем каждое семейство
    for parent_id, group in parent_groups.items():
        drugs_in_group = group['drugs']
        has_mechanism = group['has_mechanism']
        
        # Если в группе только один препарат - оставляем как есть
        if len(drugs_in_group) == 1:
            filtered_drugs.append(drugs_in_group[0])
            continue
        
        # Находим родительский препарат в группе
        parent_drug = next(
            (d for d in drugs_in_group if d['Drug ChEMBL ID'] == parent_id),
            None
        )
        
        # Если есть родительский препарат
        if parent_drug:
            # Если в семействе есть препараты с механизмом, сохраняем 'Yes' для родителя
            if has_mechanism:
                parent_drug['Drug Mechanism ChEMBL'] = 'Yes'
            filtered_drugs.append(parent_drug)
        else:
            # Если нет родительского - выбираем препарат с максимальным pChEMBL
            best_drug = max(
                drugs_in_group,
                key=lambda x: float(x['pChEMBL value']) if x.get('pChEMBL value', 'N/A') != 'N/A' else -1
            )
            filtered_drugs.append(best_drug)
    
    return filtered_drugs

def get_action_type(chembl_id: str) -> str:
    try:
        client = new_client
        mechanisms = client.mechanism.filter(molecule_chembl_id=chembl_id)
        
        action_types = set()
        for mechanism in mechanisms:
            action_type = mechanism.get("action_type")
            if action_type:
                action_types.add(action_type)
        
        return "; ".join(action_types) if action_types else "N/A"
    except Exception as e:
        print(f"Ошибка при получении типа действия для {chembl_id}: {e}")
        return "N/A"

def get_mechanism_details(chembl_id: str) -> List[Dict]:
    try:
        client = new_client
        mechanisms = client.mechanism.filter(molecule_chembl_id=chembl_id)
        
        details = []
        for mechanism in mechanisms:
            mechanism_of_action = mechanism.get("mechanism_of_action", "N/A")
            
            details.append({
                "Mechanism of Action ChEMBL": mechanism_of_action,  # Изменено здесь
            })
        
        return details if details else [{"Mechanism of Action ChEMBL": "N/A"}]  # И здесь
    except Exception as e:
        print(f"Ошибка при получении механизма действия для {chembl_id}: {e}")
        return [{"Mechanism of Action ChEMBL": "N/A"}]  # И здесь

def get_max_phase(chembl_id: str) -> str:
    try:
        client = new_client
        molecule = client.molecule.get(chembl_id)
        max_phase = molecule.get("max_phase", None)
        
        if max_phase is None:
            return ""
        
        if max_phase == "0.0":
            return "Preclinical"
        elif max_phase == "1.0":
            return "Phase 1"
        elif max_phase == "2.0":
            return "Phase 2"
        elif max_phase == "3.0":
            return "Phase 3"
        elif max_phase == "4.0":
            return "Approved"
        else:
            return f"Unknown phase ({max_phase})"
    except Exception as e:
        print(f"Ошибка при получении фазы для {chembl_id}: {e}")
        return "N/A"

def get_pchembl_value(molecule_chembl_id: str, target_chembl_id: str) -> str:
    try:
        activities = new_client.activity.filter(
            molecule_chembl_id=molecule_chembl_id,
            target_chembl_id=target_chembl_id,
            pchembl_value__isnull=False
        ).only("pchembl_value")
        return activities[0]["pchembl_value"] if activities else "N/A"
    except Exception as e:
        print(f"Error fetching pChEMBL value for {molecule_chembl_id}: {e}")
        return "N/A"

def get_activity_data(molecule_chembl_id: str, target_chembl_id: str) -> Dict:
    """Возвращает объединенные данные активности: значение + единицы измерения"""
    try:
        activities = new_client.activity.filter(
            molecule_chembl_id=molecule_chembl_id,
            target_chembl_id=target_chembl_id,
            standard_type__in=["IC50", "Kd", "Ki", "EC50"]
        ).only("standard_value", "standard_type", "standard_units")
        
        if not activities:
            return {
                "Activity value ChEMBL": "N/A",
                "Activity type ChEMBL": "N/A"
            }
        
        activity = activities[0]
        value = activity.get('standard_value')
        units = activity.get('standard_units')
        act_type = activity.get('standard_type')
        
        if act_type == "Activity":
            act_type = "N/A"
        
        if value and units and value != "N/A" and units != "N/A":
            activity_value = f"= {value} {units}"
        else:
            activity_value = "N/A"
        
        return {
            "Activity value ChEMBL": activity_value,
            "Activity type ChEMBL": act_type
        }
    except Exception as e:
        print(f"Ошибка при получении данных активности для {molecule_chembl_id}: {e}")
        return {
            "Activity value ChEMBL": "N/A",
            "Activity type ChEMBL": "N/A"
        }

def get_drugs_for_target(chembl_id: str) -> List[Dict]:
    try:
        client = new_client
        
        mechanism_drugs = []
        mechanisms = client.mechanism.filter(target_chembl_id=chembl_id)
        for mech in mechanisms:
            try:
                drug_id = mech['molecule_chembl_id']
                drug = client.molecule.get(drug_id)
                
                activity_data = get_activity_data(drug_id, chembl_id)
                mechanism_details = get_mechanism_details(drug_id)
                
                if not mechanism_details:
                    print(f"Нет данных о механизме для {drug_id}")
                    continue
                    
                mechanism_drugs.append({
                    "Drug ChEMBL ID": drug_id,
                    "Drug name": drug.get('pref_name', 'N/A'),
                    "Drug Mechanism ChEMBL": "Yes",
                    "Activity value ChEMBL": activity_data["Activity value ChEMBL"],
                    "Activity type ChEMBL": activity_data["Activity type ChEMBL"],
                    "pChEMBL value": get_pchembl_value(drug_id, chembl_id),
                    "Max phase ChEMBL": get_max_phase(drug_id),
                    "Action type ChEMBL": get_action_type(drug_id),
                    "Mechanism of Action ChEMBL": mechanism_details[0]["Mechanism of Action ChEMBL"]
                })
                time.sleep(0.1)
            except Exception as e:
                print(f"Ошибка при обработке механизма {mech.get('molecule_chembl_id')}: {str(e)}")
                continue
        
        mechanism_ids = {drug["Drug ChEMBL ID"] for drug in mechanism_drugs}
        activity_drugs = []
        
        activities = client.activity.filter(
            target_chembl_id=chembl_id,
            pchembl_value__isnull=False,
            standard_type__in=["IC50", "Kd", "Ki", "EC50"]
        ).order_by('-pchembl_value')[:100]
        
        for activity in activities:
            try:
                if activity['molecule_chembl_id'] not in mechanism_ids:
                    value = activity.get('standard_value', 'N/A')
                    units = activity.get('standard_units', 'N/A')
                    act_type = activity.get('standard_type', 'N/A')
                    
                    if act_type == "Activity":
                        act_type = "N/A"
                    
                    activity_value = f"= {value} {units}" if value != "N/A" and units != "N/A" else "N/A"
                    
                    activity_drugs.append({
                        "Drug ChEMBL ID": activity['molecule_chembl_id'],
                        "Drug name": activity['molecule_pref_name'],
                        "Drug Mechanism ChEMBL": "No",
                        "Activity value ChEMBL": activity_value,  # Исправлено
                        "Activity type ChEMBL": act_type,          # Исправлено
                        "pChEMBL value": activity['pchembl_value'],
                        "Max phase ChEMBL": get_max_phase(activity['molecule_chembl_id']),  # Исправлено
                        "Action type ChEMBL": get_action_type(activity['molecule_chembl_id']),  # Исправлено
                        "Mechanism of Action ChEMBL": "N/A"
                    })
                    time.sleep(0.1)
            except Exception as e:
                print(f"Ошибка при обработке активности {activity}: {e}")
                continue
        
        all_drugs = mechanism_drugs + activity_drugs
        return filter_parent_drugs(all_drugs)
    except Exception as e:
        print(f"Ошибка при получении препаратов для цели {chembl_id}: {e}")
        return []

def get_gene_drug_info(gene_names: List[str]) -> List[Dict]:
    result = []
    
    for gene_name in gene_names:
        try:
            print(f"Обрабатывается ген: {gene_name}")
            gene_info_list = search_gene_in_uniprot_filtered(gene_name)
            
            if isinstance(gene_info_list, str):
                result.append({"Gene": gene_name, "Error": gene_info_list})
                continue
            
            unique_drugs_for_gene = {}

            for gene_info in gene_info_list:
                try:
                    if isinstance(gene_info['ChEMBL_ID'], str) and gene_info['ChEMBL_ID'].startswith("No ChEMBL"):
                        continue
                        
                    drugs = get_drugs_for_target(gene_info['ChEMBL_ID'])  # Только 1 аргумент!
                    
                    for drug in drugs:
                        chembl_id = drug["Drug ChEMBL ID"]
                        if chembl_id not in unique_drugs_for_gene or unique_drugs_for_gene[chembl_id]["pChEMBL value"] < drug["pChEMBL value"]:
                            unique_drugs_for_gene[chembl_id] = drug
                except Exception as e:
                    print(f"Ошибка при обработке информации о гене {gene_name}: {e}")
                    continue
            
            if not gene_info_list or 'UniProt_ID' not in gene_info_list[0]:
                continue
                
            for drug in unique_drugs_for_gene.values():
                result.append({
                    "Gene": gene_name,
                    "UniProt ID": gene_info_list[0]['UniProt_ID'],
                    "Protein name": gene_info_list[0]['Protein_Name'],
                    "Protein ChEMBL ID": gene_info_list[0]['ChEMBL_ID'],
                    **drug
                })
                
        except Exception as e:
            print(f"Ошибка при обработке гена {gene_name}: {e}")
            result.append({"Gene": gene_name, "Error": str(e)})
            continue

    return result

if __name__ == "__main__":
    # Замените этот путь на ваш реальный путь к файлу
    input_file = r"C:\Users\rusla\OneDrive\Рабочий стол\Диплом\Гены\thresholds\gene_selection\ALL\ALL_depscore_-0.2_zscore_-1.5.csv"
    
    # Чтение генов из файла
    try:
        genes_df = pd.read_csv(input_file)
        gene_names = genes_df['Gene'].tolist()
        print(f"Загружено {len(gene_names)} генов из файла")
    except Exception as e:
        print(f"Ошибка при чтении файла {input_file}: {e}")
        raise
    
    # Получаем имя выходного файла (без пути)
    output_filename = os.path.basename(input_file)
    
    start_time = time.time()
    print("\nНачинаем обработку с динамической фильтрацией pChEMBL")
    
    try:
        gene_drug_info = get_gene_drug_info(gene_names)
        
        # Определяем полный список всех возможных колонок
        all_columns = [
            "Gene", "UniProt ID", "Protein name", "Protein ChEMBL ID",
            "Drug ChEMBL ID", "Drug name", "Drug Mechanism ChEMBL",
            "Activity type ChEMBL", "Activity value ChEMBL",
            "pChEMBL value", "Max phase ChEMBL", "Action type ChEMBL", 
            "Mechanism of Action ChEMBL"
        ]
        
        # Создаем DataFrame из результатов
        df = pd.DataFrame(gene_drug_info)
        
        # Добавляем отсутствующие колонки и заполняем их NA
        for col in all_columns:
            if col not in df.columns:
                df[col] = pd.NA
        
        # Упорядочиваем колонки согласно нашему списку
        df = df[all_columns]
        
        # Фильтруем ошибочные записи (где нет Drug ChEMBL ID)
        df = df[df['Drug ChEMBL ID'].notna()]
        
        # Сохраняем в текущую директорию с тем же именем файла
        df.to_csv(output_filename, index=False, encoding='utf-8')
        
        elapsed_time = time.time() - start_time
        
        # Собираем статистику
        num_drugs = len(df)
        num_mechanisms = len(df[df['Drug Mechanism ChEMBL'] == 'Yes'])
        
        print(f"Файл {output_filename} сохранён. Время выполнения: {elapsed_time:.2f} секунд")
        print(f"Статистика: {num_drugs} препаратов, из них {num_mechanisms} с известным механизмом")
        
    except Exception as e:
        print(f"Критическая ошибка при обработке: {e}")

Загружено 46 генов из файла

Начинаем обработку с динамической фильтрацией pChEMBL
Обрабатывается ген: ARHGAP45
Обрабатывается ген: ATP1B3
Обрабатывается ген: ATP6V0A2
Обрабатывается ген: BCL2
Обрабатывается ген: C10orf95
Обрабатывается ген: C16orf52
Обрабатывается ген: CARS2
Обрабатывается ген: CBFB
Обрабатывается ген: CCND3
Обрабатывается ген: COQ4
Обрабатывается ген: DHODH
Обрабатывается ген: EBF1
Обрабатывается ген: ERG
Обрабатывается ген: FAM19A4
Обрабатывается ген: FAM214B
Обрабатывается ген: FAR1
Обрабатывается ген: FLJ42393
Обрабатывается ген: GCLC
Обрабатывается ген: IKZF1
Обрабатывается ген: LCE1F
Обрабатывается ген: LEF1
Обрабатывается ген: LOC107984065
Обрабатывается ген: LOC107987188
Обрабатывается ген: LOC440434
Обрабатывается ген: MAGEA3
Обрабатывается ген: MBNL1
Обрабатывается ген: MEF2C
Обрабатывается ген: MIR429
Обрабатывается ген: MON1A
Обрабатывается ген: MSL3P1
Обрабатывается ген: MYB
Обрабатывается ген: NMNAT1
Обрабатывается ген: PAICS
Обрабатывается ген: PAX5
Обр

In [14]:
import requests
from chembl_webresource_client.new_client import new_client
import pandas as pd
import numpy as np
from typing import List, Dict, Union, Optional
from functools import lru_cache
import time
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
import os
from pathlib import Path

# Настройка сессии с повторными попытками
session = requests.Session()
retries = Retry(
    total=5,
    backoff_factor=1,
    status_forcelist=[500, 502, 503, 504]
)
session.mount('https://', HTTPAdapter(max_retries=retries))

# Установка таймаута по умолчанию для ChEMBL клиента
new_client.timeout = 30

def uniprot_to_chembl(uniprot_id: str) -> Optional[str]:
    """Получает ChEMBL ID из UniProt ID."""
    target = new_client.target
    try:
        res = target.filter(target_components__accession=uniprot_id)
        res_df = pd.DataFrame(res)
        
        if res_df.empty:
            return None
        return res_df['target_chembl_id'].iloc[0]
    except Exception as e:
        print(f"Ошибка при получении ChEMBL ID для UniProt {uniprot_id}: {e}")
        return None

def search_gene_in_uniprot_filtered(gene_name: str, organism: str = "Homo sapiens") -> Union[str, List[Dict]]:
    """Ищет информацию о гене в UniProt."""
    url = "https://rest.uniprot.org/uniprotkb/search"
    query = f'gene_exact:"{gene_name}" AND reviewed:true AND taxonomy_name:"{organism}"'
    
    params = {
        "query": query,
        "format": "json",
        "fields": "accession,gene_names,protein_name",
        "size": 10
    }
    
    try:
        response = session.get(url, params=params, timeout=30)
        response.raise_for_status()
        
        results = response.json().get("results", [])
        if not results:
            return f"No results found for gene '{gene_name}'"
        
        gene_info = [
            {
                "UniProt_ID": result["primaryAccession"],
                "Gene_Names": result["genes"][0]["geneName"]["value"] if result.get("genes") else "N/A",
                "Protein_Name": result["proteinDescription"]["recommendedName"]["fullName"]["value"],
            }
            for result in results
        ]
        
        for gene in gene_info:
            uniprot_id = gene["UniProt_ID"]
            chembl_id = uniprot_to_chembl(uniprot_id)
            gene["ChEMBL_ID"] = chembl_id if chembl_id else f"No ChEMBL ID found for {uniprot_id}"
            time.sleep(0.1)  # Задержка между запросами
        
        return gene_info
    except Exception as e:
        print(f"Ошибка при поиске гена {gene_name}: {e}")
        return f"Error searching gene {gene_name}: {str(e)}"

@lru_cache(maxsize=1000)
def get_parent_info(chembl_id: str) -> dict:
    """Получает информацию о родительском соединении для данного ChEMBL ID."""
    molecule = new_client.molecule
    molecule_form = new_client.molecule_form
    
    try:
        # Проверяем, есть ли родительское соединение
        parent_data = list(molecule_form.filter(molecule_chembl_id=chembl_id).only('parent_chembl_id'))
        
        if parent_data and 'parent_chembl_id' in parent_data[0]:
            parent_id = parent_data[0]['parent_chembl_id']
        else:
            # Если нет родителя, считаем текущий ID родительским
            parent_id = chembl_id
        
        # Получаем информацию о родительском соединении
        mol_data = list(molecule.filter(molecule_chembl_id=parent_id).only('pref_name', 'molecule_chembl_id'))
        pref_name = mol_data[0].get('pref_name', 'Unknown') if mol_data else 'Unknown'
        
        return {
            'parent_chembl_id': parent_id,
            'parent_pref_name': pref_name,
            'is_parent': parent_id == chembl_id
        }
    except Exception as e:
        print(f"Error getting parent info for {chembl_id}: {e}")
        return {
            'parent_chembl_id': chembl_id,
            'parent_pref_name': 'Unknown',
            'is_parent': True
        }

def filter_parent_drugs(drugs_list: List[Dict]) -> List[Dict]:
    """Фильтрует список препаратов, оставляя по одному представителю из каждого семейства.
       Сохраняет 'Yes' в Drug Mechanism при замене дочернего с 'Yes' на родительский."""
    parent_groups = {}
    
    # 1. Группируем все препараты по семействам
    for drug in drugs_list:
        try:
            if not drug.get('Drug ChEMBL ID'):
                continue
                
            parent_info = get_parent_info(drug['Drug ChEMBL ID'])
            parent_id = parent_info['parent_chembl_id']
            
            if parent_id not in parent_groups:
                parent_groups[parent_id] = {
                    'parent_info': parent_info,
                    'drugs': [],
                    'has_mechanism': False
                }
            
            parent_groups[parent_id]['drugs'].append(drug)
            if drug.get('Drug Mechanism ChEMBL') == 'Yes':
                parent_groups[parent_id]['has_mechanism'] = True
                
        except Exception as e:
            print(f"Ошибка при группировке препаратов: {e}")
            continue
    
    filtered_drugs = []
    
    # 2. Обрабатываем каждое семейство
    for parent_id, group in parent_groups.items():
        drugs_in_group = group['drugs']
        has_mechanism = group['has_mechanism']
        
        # Если в группе только один препарат - оставляем как есть
        if len(drugs_in_group) == 1:
            filtered_drugs.append(drugs_in_group[0])
            continue
        
        # Находим родительский препарат в группе
        parent_drug = next(
            (d for d in drugs_in_group if d['Drug ChEMBL ID'] == parent_id),
            None
        )
        
        # Если есть родительский препарат
        if parent_drug:
            # Если в семействе есть препараты с механизмом, сохраняем 'Yes' для родителя
            if has_mechanism:
                parent_drug['Drug Mechanism ChEMBL'] = 'Yes'
            filtered_drugs.append(parent_drug)
        else:
            # Если нет родительского - выбираем препарат с максимальным pChEMBL
            best_drug = max(
                drugs_in_group,
                key=lambda x: float(x['pChEMBL value']) if x.get('pChEMBL value', 'N/A') != 'N/A' else -1
            )
            filtered_drugs.append(best_drug)
    
    return filtered_drugs

def get_action_type(chembl_id: str) -> str:
    try:
        client = new_client
        mechanisms = client.mechanism.filter(molecule_chembl_id=chembl_id)
        
        action_types = set()
        for mechanism in mechanisms:
            action_type = mechanism.get("action_type")
            if action_type:
                action_types.add(action_type)
        
        return "; ".join(action_types) if action_types else "N/A"
    except Exception as e:
        print(f"Ошибка при получении типа действия для {chembl_id}: {e}")
        return "N/A"

def get_mechanism_details(chembl_id: str) -> List[Dict]:
    try:
        client = new_client
        mechanisms = client.mechanism.filter(molecule_chembl_id=chembl_id)
        
        details = []
        for mechanism in mechanisms:
            mechanism_of_action = mechanism.get("mechanism_of_action", "N/A")
            
            details.append({
                "Mechanism of Action ChEMBL": mechanism_of_action,  # Изменено здесь
            })
        
        return details if details else [{"Mechanism of Action ChEMBL": "N/A"}]  # И здесь
    except Exception as e:
        print(f"Ошибка при получении механизма действия для {chembl_id}: {e}")
        return [{"Mechanism of Action ChEMBL": "N/A"}]  # И здесь

def get_max_phase(chembl_id: str) -> str:
    try:
        client = new_client
        molecule = client.molecule.get(chembl_id)
        max_phase = molecule.get("max_phase", None)
        
        if max_phase is None:
            return ""
        
        if max_phase == "0.0":
            return "Preclinical"
        elif max_phase == "1.0":
            return "Phase 1"
        elif max_phase == "2.0":
            return "Phase 2"
        elif max_phase == "3.0":
            return "Phase 3"
        elif max_phase == "4.0":
            return "Approved"
        else:
            return f"Unknown phase ({max_phase})"
    except Exception as e:
        print(f"Ошибка при получении фазы для {chembl_id}: {e}")
        return "N/A"

def get_pchembl_value(molecule_chembl_id: str, target_chembl_id: str) -> str:
    try:
        activities = new_client.activity.filter(
            molecule_chembl_id=molecule_chembl_id,
            target_chembl_id=target_chembl_id,
            pchembl_value__isnull=False
        ).only("pchembl_value")
        return activities[0]["pchembl_value"] if activities else "N/A"
    except Exception as e:
        print(f"Error fetching pChEMBL value for {molecule_chembl_id}: {e}")
        return "N/A"

def get_activity_data(molecule_chembl_id: str, target_chembl_id: str) -> Dict:
    """Возвращает объединенные данные активности: значение + единицы измерения"""
    try:
        activities = new_client.activity.filter(
            molecule_chembl_id=molecule_chembl_id,
            target_chembl_id=target_chembl_id,
            standard_type__in=["IC50", "Kd", "Ki", "EC50"]
        ).only("standard_value", "standard_type", "standard_units")
        
        if not activities:
            return {
                "Activity value ChEMBL": "N/A",
                "Activity type ChEMBL": "N/A"
            }
        
        activity = activities[0]
        value = activity.get('standard_value')
        units = activity.get('standard_units')
        act_type = activity.get('standard_type')
        
        if act_type == "Activity":
            act_type = "N/A"
        
        if value and units and value != "N/A" and units != "N/A":
            activity_value = f"= {value} {units}"
        else:
            activity_value = "N/A"
        
        return {
            "Activity value ChEMBL": activity_value,
            "Activity type ChEMBL": act_type
        }
    except Exception as e:
        print(f"Ошибка при получении данных активности для {molecule_chembl_id}: {e}")
        return {
            "Activity value ChEMBL": "N/A",
            "Activity type ChEMBL": "N/A"
        }

def get_drugs_for_target(chembl_id: str) -> List[Dict]:
    try:
        client = new_client
        
        mechanism_drugs = []
        mechanisms = client.mechanism.filter(target_chembl_id=chembl_id)
        for mech in mechanisms:
            try:
                drug_id = mech['molecule_chembl_id']
                drug = client.molecule.get(drug_id)
                
                activity_data = get_activity_data(drug_id, chembl_id)
                mechanism_details = get_mechanism_details(drug_id)
                
                if not mechanism_details:
                    print(f"Нет данных о механизме для {drug_id}")
                    continue
                    
                mechanism_drugs.append({
                    "Drug ChEMBL ID": drug_id,
                    "Drug name": drug.get('pref_name', 'N/A'),
                    "Drug Mechanism ChEMBL": "Yes",
                    "Activity value ChEMBL": activity_data["Activity value ChEMBL"],
                    "Activity type ChEMBL": activity_data["Activity type ChEMBL"],
                    "pChEMBL value": get_pchembl_value(drug_id, chembl_id),
                    "Max phase ChEMBL": get_max_phase(drug_id),
                    "Action type ChEMBL": get_action_type(drug_id),
                    "Mechanism of Action ChEMBL": mechanism_details[0]["Mechanism of Action ChEMBL"]
                })
                time.sleep(0.1)
            except Exception as e:
                print(f"Ошибка при обработке механизма {mech.get('molecule_chembl_id')}: {str(e)}")
                continue
        
        mechanism_ids = {drug["Drug ChEMBL ID"] for drug in mechanism_drugs}
        activity_drugs = []
        
        activities = client.activity.filter(
            target_chembl_id=chembl_id,
            pchembl_value__isnull=False,
            standard_type__in=["IC50", "Kd", "Ki", "EC50"]
        ).order_by('-pchembl_value')[:100]
        
        for activity in activities:
            try:
                if activity['molecule_chembl_id'] not in mechanism_ids:
                    value = activity.get('standard_value', 'N/A')
                    units = activity.get('standard_units', 'N/A')
                    act_type = activity.get('standard_type', 'N/A')
                    
                    if act_type == "Activity":
                        act_type = "N/A"
                    
                    activity_value = f"= {value} {units}" if value != "N/A" and units != "N/A" else "N/A"
                    
                    activity_drugs.append({
                        "Drug ChEMBL ID": activity['molecule_chembl_id'],
                        "Drug name": activity['molecule_pref_name'],
                        "Drug Mechanism ChEMBL": "No",
                        "Activity value ChEMBL": activity_value,  # Исправлено
                        "Activity type ChEMBL": act_type,          # Исправлено
                        "pChEMBL value": activity['pchembl_value'],
                        "Max phase ChEMBL": get_max_phase(activity['molecule_chembl_id']),  # Исправлено
                        "Action type ChEMBL": get_action_type(activity['molecule_chembl_id']),  # Исправлено
                        "Mechanism of Action ChEMBL": "N/A"
                    })
                    time.sleep(0.1)
            except Exception as e:
                print(f"Ошибка при обработке активности {activity}: {e}")
                continue
        
        all_drugs = mechanism_drugs + activity_drugs
        return filter_parent_drugs(all_drugs)
    except Exception as e:
        print(f"Ошибка при получении препаратов для цели {chembl_id}: {e}")
        return []

def get_gene_drug_info(gene_names: List[str]) -> List[Dict]:
    result = []
    
    for gene_name in gene_names:
        try:
            print(f"Обрабатывается ген: {gene_name}")
            gene_info_list = search_gene_in_uniprot_filtered(gene_name)
            
            if isinstance(gene_info_list, str):
                result.append({"Gene": gene_name, "Error": gene_info_list})
                continue
            
            unique_drugs_for_gene = {}

            for gene_info in gene_info_list:
                try:
                    if isinstance(gene_info['ChEMBL_ID'], str) and gene_info['ChEMBL_ID'].startswith("No ChEMBL"):
                        continue
                        
                    drugs = get_drugs_for_target(gene_info['ChEMBL_ID'])  # Только 1 аргумент!
                    
                    for drug in drugs:
                        chembl_id = drug["Drug ChEMBL ID"]
                        if chembl_id not in unique_drugs_for_gene or unique_drugs_for_gene[chembl_id]["pChEMBL value"] < drug["pChEMBL value"]:
                            unique_drugs_for_gene[chembl_id] = drug
                except Exception as e:
                    print(f"Ошибка при обработке информации о гене {gene_name}: {e}")
                    continue
            
            if not gene_info_list or 'UniProt_ID' not in gene_info_list[0]:
                continue
                
            for drug in unique_drugs_for_gene.values():
                result.append({
                    "Gene": gene_name,
                    "UniProt ID": gene_info_list[0]['UniProt_ID'],
                    "Protein name": gene_info_list[0]['Protein_Name'],
                    "Protein ChEMBL ID": gene_info_list[0]['ChEMBL_ID'],
                    **drug
                })
                
        except Exception as e:
            print(f"Ошибка при обработке гена {gene_name}: {e}")
            result.append({"Gene": gene_name, "Error": str(e)})
            continue

    return result

if __name__ == "__main__":
    # Замените этот путь на ваш реальный путь к файлу
    input_file = r"C:\Users\rusla\OneDrive\Рабочий стол\Диплом\Гены\thresholds\gene_selection\ALL\ALL_depscore_-0.2_zscore_-1.csv"
    
    # Чтение генов из файла
    try:
        genes_df = pd.read_csv(input_file)
        gene_names = genes_df['Gene'].tolist()
        print(f"Загружено {len(gene_names)} генов из файла")
    except Exception as e:
        print(f"Ошибка при чтении файла {input_file}: {e}")
        raise
    
    # Получаем имя выходного файла (без пути)
    output_filename = os.path.basename(input_file)
    
    start_time = time.time()
    print("\nНачинаем обработку с динамической фильтрацией pChEMBL")
    
    try:
        gene_drug_info = get_gene_drug_info(gene_names)
        
        # Определяем полный список всех возможных колонок
        all_columns = [
            "Gene", "UniProt ID", "Protein name", "Protein ChEMBL ID",
            "Drug ChEMBL ID", "Drug name", "Drug Mechanism ChEMBL",
            "Activity type ChEMBL", "Activity value ChEMBL",
            "pChEMBL value", "Max phase ChEMBL", "Action type ChEMBL", 
            "Mechanism of Action ChEMBL"
        ]
        
        # Создаем DataFrame из результатов
        df = pd.DataFrame(gene_drug_info)
        
        # Добавляем отсутствующие колонки и заполняем их NA
        for col in all_columns:
            if col not in df.columns:
                df[col] = pd.NA
        
        # Упорядочиваем колонки согласно нашему списку
        df = df[all_columns]
        
        # Фильтруем ошибочные записи (где нет Drug ChEMBL ID)
        df = df[df['Drug ChEMBL ID'].notna()]
        
        # Сохраняем в текущую директорию с тем же именем файла
        df.to_csv(output_filename, index=False, encoding='utf-8')
        
        elapsed_time = time.time() - start_time
        
        # Собираем статистику
        num_drugs = len(df)
        num_mechanisms = len(df[df['Drug Mechanism ChEMBL'] == 'Yes'])
        
        print(f"Файл {output_filename} сохранён. Время выполнения: {elapsed_time:.2f} секунд")
        print(f"Статистика: {num_drugs} препаратов, из них {num_mechanisms} с известным механизмом")
        
    except Exception as e:
        print(f"Критическая ошибка при обработке: {e}")

Загружено 224 генов из файла

Начинаем обработку с динамической фильтрацией pChEMBL
Обрабатывается ген: ADSS2
Обрабатывается ген: AES
Обрабатывается ген: AGPS
Обрабатывается ген: AHCY
Обрабатывается ген: AHDC1
Обрабатывается ген: AK2
Обрабатывается ген: ALAD
Обрабатывается ген: ALAS1
Обрабатывается ген: ANKH
Обрабатывается ген: ANKRD36
Обрабатывается ген: ARHGAP19-SLIT1
Обрабатывается ген: ARHGAP45
Обрабатывается ген: ATIC
Обрабатывается ген: ATP1B3
Обрабатывается ген: ATP6V0A2
Обрабатывается ген: BCL2
Обрабатывается ген: BCS1L
Обрабатывается ген: BIRC8
Обрабатывается ген: C10orf95
Обрабатывается ген: C14orf2
Обрабатывается ген: C16orf52
Обрабатывается ген: C1QTNF5&MFRP
Обрабатывается ген: C1orf228
Обрабатывается ген: C8orf44-SGK3
Обрабатывается ген: CAB39
Обрабатывается ген: CAD
Обрабатывается ген: CARS2
Обрабатывается ген: CBFB
Обрабатывается ген: CCM2
Обрабатывается ген: CCND3
Обрабатывается ген: CCNYL2
Обрабатывается ген: CDIN1
Обрабатывается ген: CDK6
Обрабатывается ген: CEND1
Обр

In [16]:
import requests
from chembl_webresource_client.new_client import new_client
import pandas as pd
import numpy as np
from typing import List, Dict, Union, Optional
from functools import lru_cache
import time
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
import os
from pathlib import Path

# Настройка сессии с повторными попытками
session = requests.Session()
retries = Retry(
    total=5,
    backoff_factor=1,
    status_forcelist=[500, 502, 503, 504]
)
session.mount('https://', HTTPAdapter(max_retries=retries))

# Установка таймаута по умолчанию для ChEMBL клиента
new_client.timeout = 30

def uniprot_to_chembl(uniprot_id: str) -> Optional[str]:
    """Получает ChEMBL ID из UniProt ID."""
    target = new_client.target
    try:
        res = target.filter(target_components__accession=uniprot_id)
        res_df = pd.DataFrame(res)
        
        if res_df.empty:
            return None
        return res_df['target_chembl_id'].iloc[0]
    except Exception as e:
        print(f"Ошибка при получении ChEMBL ID для UniProt {uniprot_id}: {e}")
        return None

def search_gene_in_uniprot_filtered(gene_name: str, organism: str = "Homo sapiens") -> Union[str, List[Dict]]:
    """Ищет информацию о гене в UniProt."""
    url = "https://rest.uniprot.org/uniprotkb/search"
    query = f'gene_exact:"{gene_name}" AND reviewed:true AND taxonomy_name:"{organism}"'
    
    params = {
        "query": query,
        "format": "json",
        "fields": "accession,gene_names,protein_name",
        "size": 10
    }
    
    try:
        response = session.get(url, params=params, timeout=30)
        response.raise_for_status()
        
        results = response.json().get("results", [])
        if not results:
            return f"No results found for gene '{gene_name}'"
        
        gene_info = [
            {
                "UniProt_ID": result["primaryAccession"],
                "Gene_Names": result["genes"][0]["geneName"]["value"] if result.get("genes") else "N/A",
                "Protein_Name": result["proteinDescription"]["recommendedName"]["fullName"]["value"],
            }
            for result in results
        ]
        
        for gene in gene_info:
            uniprot_id = gene["UniProt_ID"]
            chembl_id = uniprot_to_chembl(uniprot_id)
            gene["ChEMBL_ID"] = chembl_id if chembl_id else f"No ChEMBL ID found for {uniprot_id}"
            time.sleep(0.1)  # Задержка между запросами
        
        return gene_info
    except Exception as e:
        print(f"Ошибка при поиске гена {gene_name}: {e}")
        return f"Error searching gene {gene_name}: {str(e)}"

@lru_cache(maxsize=1000)
def get_parent_info(chembl_id: str) -> dict:
    """Получает информацию о родительском соединении для данного ChEMBL ID."""
    molecule = new_client.molecule
    molecule_form = new_client.molecule_form
    
    try:
        # Проверяем, есть ли родительское соединение
        parent_data = list(molecule_form.filter(molecule_chembl_id=chembl_id).only('parent_chembl_id'))
        
        if parent_data and 'parent_chembl_id' in parent_data[0]:
            parent_id = parent_data[0]['parent_chembl_id']
        else:
            # Если нет родителя, считаем текущий ID родительским
            parent_id = chembl_id
        
        # Получаем информацию о родительском соединении
        mol_data = list(molecule.filter(molecule_chembl_id=parent_id).only('pref_name', 'molecule_chembl_id'))
        pref_name = mol_data[0].get('pref_name', 'Unknown') if mol_data else 'Unknown'
        
        return {
            'parent_chembl_id': parent_id,
            'parent_pref_name': pref_name,
            'is_parent': parent_id == chembl_id
        }
    except Exception as e:
        print(f"Error getting parent info for {chembl_id}: {e}")
        return {
            'parent_chembl_id': chembl_id,
            'parent_pref_name': 'Unknown',
            'is_parent': True
        }

def filter_parent_drugs(drugs_list: List[Dict]) -> List[Dict]:
    """Фильтрует список препаратов, оставляя по одному представителю из каждого семейства.
       Сохраняет 'Yes' в Drug Mechanism при замене дочернего с 'Yes' на родительский."""
    parent_groups = {}
    
    # 1. Группируем все препараты по семействам
    for drug in drugs_list:
        try:
            if not drug.get('Drug ChEMBL ID'):
                continue
                
            parent_info = get_parent_info(drug['Drug ChEMBL ID'])
            parent_id = parent_info['parent_chembl_id']
            
            if parent_id not in parent_groups:
                parent_groups[parent_id] = {
                    'parent_info': parent_info,
                    'drugs': [],
                    'has_mechanism': False
                }
            
            parent_groups[parent_id]['drugs'].append(drug)
            if drug.get('Drug Mechanism ChEMBL') == 'Yes':
                parent_groups[parent_id]['has_mechanism'] = True
                
        except Exception as e:
            print(f"Ошибка при группировке препаратов: {e}")
            continue
    
    filtered_drugs = []
    
    # 2. Обрабатываем каждое семейство
    for parent_id, group in parent_groups.items():
        drugs_in_group = group['drugs']
        has_mechanism = group['has_mechanism']
        
        # Если в группе только один препарат - оставляем как есть
        if len(drugs_in_group) == 1:
            filtered_drugs.append(drugs_in_group[0])
            continue
        
        # Находим родительский препарат в группе
        parent_drug = next(
            (d for d in drugs_in_group if d['Drug ChEMBL ID'] == parent_id),
            None
        )
        
        # Если есть родительский препарат
        if parent_drug:
            # Если в семействе есть препараты с механизмом, сохраняем 'Yes' для родителя
            if has_mechanism:
                parent_drug['Drug Mechanism ChEMBL'] = 'Yes'
            filtered_drugs.append(parent_drug)
        else:
            # Если нет родительского - выбираем препарат с максимальным pChEMBL
            best_drug = max(
                drugs_in_group,
                key=lambda x: float(x['pChEMBL value']) if x.get('pChEMBL value', 'N/A') != 'N/A' else -1
            )
            filtered_drugs.append(best_drug)
    
    return filtered_drugs

def get_action_type(chembl_id: str) -> str:
    try:
        client = new_client
        mechanisms = client.mechanism.filter(molecule_chembl_id=chembl_id)
        
        action_types = set()
        for mechanism in mechanisms:
            action_type = mechanism.get("action_type")
            if action_type:
                action_types.add(action_type)
        
        return "; ".join(action_types) if action_types else "N/A"
    except Exception as e:
        print(f"Ошибка при получении типа действия для {chembl_id}: {e}")
        return "N/A"

def get_mechanism_details(chembl_id: str) -> List[Dict]:
    try:
        client = new_client
        mechanisms = client.mechanism.filter(molecule_chembl_id=chembl_id)
        
        details = []
        for mechanism in mechanisms:
            mechanism_of_action = mechanism.get("mechanism_of_action", "N/A")
            
            details.append({
                "Mechanism of Action ChEMBL": mechanism_of_action,  # Изменено здесь
            })
        
        return details if details else [{"Mechanism of Action ChEMBL": "N/A"}]  # И здесь
    except Exception as e:
        print(f"Ошибка при получении механизма действия для {chembl_id}: {e}")
        return [{"Mechanism of Action ChEMBL": "N/A"}]  # И здесь

def get_max_phase(chembl_id: str) -> str:
    try:
        client = new_client
        molecule = client.molecule.get(chembl_id)
        max_phase = molecule.get("max_phase", None)
        
        if max_phase is None:
            return ""
        
        if max_phase == "0.0":
            return "Preclinical"
        elif max_phase == "1.0":
            return "Phase 1"
        elif max_phase == "2.0":
            return "Phase 2"
        elif max_phase == "3.0":
            return "Phase 3"
        elif max_phase == "4.0":
            return "Approved"
        else:
            return f"Unknown phase ({max_phase})"
    except Exception as e:
        print(f"Ошибка при получении фазы для {chembl_id}: {e}")
        return "N/A"

def get_pchembl_value(molecule_chembl_id: str, target_chembl_id: str) -> str:
    try:
        activities = new_client.activity.filter(
            molecule_chembl_id=molecule_chembl_id,
            target_chembl_id=target_chembl_id,
            pchembl_value__isnull=False
        ).only("pchembl_value")
        return activities[0]["pchembl_value"] if activities else "N/A"
    except Exception as e:
        print(f"Error fetching pChEMBL value for {molecule_chembl_id}: {e}")
        return "N/A"

def get_activity_data(molecule_chembl_id: str, target_chembl_id: str) -> Dict:
    """Возвращает объединенные данные активности: значение + единицы измерения"""
    try:
        activities = new_client.activity.filter(
            molecule_chembl_id=molecule_chembl_id,
            target_chembl_id=target_chembl_id,
            standard_type__in=["IC50", "Kd", "Ki", "EC50"]
        ).only("standard_value", "standard_type", "standard_units")
        
        if not activities:
            return {
                "Activity value ChEMBL": "N/A",
                "Activity type ChEMBL": "N/A"
            }
        
        activity = activities[0]
        value = activity.get('standard_value')
        units = activity.get('standard_units')
        act_type = activity.get('standard_type')
        
        if act_type == "Activity":
            act_type = "N/A"
        
        if value and units and value != "N/A" and units != "N/A":
            activity_value = f"= {value} {units}"
        else:
            activity_value = "N/A"
        
        return {
            "Activity value ChEMBL": activity_value,
            "Activity type ChEMBL": act_type
        }
    except Exception as e:
        print(f"Ошибка при получении данных активности для {molecule_chembl_id}: {e}")
        return {
            "Activity value ChEMBL": "N/A",
            "Activity type ChEMBL": "N/A"
        }

def get_drugs_for_target(chembl_id: str) -> List[Dict]:
    try:
        client = new_client
        
        mechanism_drugs = []
        mechanisms = client.mechanism.filter(target_chembl_id=chembl_id)
        for mech in mechanisms:
            try:
                drug_id = mech['molecule_chembl_id']
                drug = client.molecule.get(drug_id)
                
                activity_data = get_activity_data(drug_id, chembl_id)
                mechanism_details = get_mechanism_details(drug_id)
                
                if not mechanism_details:
                    print(f"Нет данных о механизме для {drug_id}")
                    continue
                    
                mechanism_drugs.append({
                    "Drug ChEMBL ID": drug_id,
                    "Drug name": drug.get('pref_name', 'N/A'),
                    "Drug Mechanism ChEMBL": "Yes",
                    "Activity value ChEMBL": activity_data["Activity value ChEMBL"],
                    "Activity type ChEMBL": activity_data["Activity type ChEMBL"],
                    "pChEMBL value": get_pchembl_value(drug_id, chembl_id),
                    "Max phase ChEMBL": get_max_phase(drug_id),
                    "Action type ChEMBL": get_action_type(drug_id),
                    "Mechanism of Action ChEMBL": mechanism_details[0]["Mechanism of Action ChEMBL"]
                })
                time.sleep(0.1)
            except Exception as e:
                print(f"Ошибка при обработке механизма {mech.get('molecule_chembl_id')}: {str(e)}")
                continue
        
        mechanism_ids = {drug["Drug ChEMBL ID"] for drug in mechanism_drugs}
        activity_drugs = []
        
        activities = client.activity.filter(
            target_chembl_id=chembl_id,
            pchembl_value__isnull=False,
            standard_type__in=["IC50", "Kd", "Ki", "EC50"]
        ).order_by('-pchembl_value')[:100]
        
        for activity in activities:
            try:
                if activity['molecule_chembl_id'] not in mechanism_ids:
                    value = activity.get('standard_value', 'N/A')
                    units = activity.get('standard_units', 'N/A')
                    act_type = activity.get('standard_type', 'N/A')
                    
                    if act_type == "Activity":
                        act_type = "N/A"
                    
                    activity_value = f"= {value} {units}" if value != "N/A" and units != "N/A" else "N/A"
                    
                    activity_drugs.append({
                        "Drug ChEMBL ID": activity['molecule_chembl_id'],
                        "Drug name": activity['molecule_pref_name'],
                        "Drug Mechanism ChEMBL": "No",
                        "Activity value ChEMBL": activity_value,  # Исправлено
                        "Activity type ChEMBL": act_type,          # Исправлено
                        "pChEMBL value": activity['pchembl_value'],
                        "Max phase ChEMBL": get_max_phase(activity['molecule_chembl_id']),  # Исправлено
                        "Action type ChEMBL": get_action_type(activity['molecule_chembl_id']),  # Исправлено
                        "Mechanism of Action ChEMBL": "N/A"
                    })
                    time.sleep(0.1)
            except Exception as e:
                print(f"Ошибка при обработке активности {activity}: {e}")
                continue
        
        all_drugs = mechanism_drugs + activity_drugs
        return filter_parent_drugs(all_drugs)
    except Exception as e:
        print(f"Ошибка при получении препаратов для цели {chembl_id}: {e}")
        return []

def get_gene_drug_info(gene_names: List[str]) -> List[Dict]:
    result = []
    
    for gene_name in gene_names:
        try:
            print(f"Обрабатывается ген: {gene_name}")
            gene_info_list = search_gene_in_uniprot_filtered(gene_name)
            
            if isinstance(gene_info_list, str):
                result.append({"Gene": gene_name, "Error": gene_info_list})
                continue
            
            unique_drugs_for_gene = {}

            for gene_info in gene_info_list:
                try:
                    if isinstance(gene_info['ChEMBL_ID'], str) and gene_info['ChEMBL_ID'].startswith("No ChEMBL"):
                        continue
                        
                    drugs = get_drugs_for_target(gene_info['ChEMBL_ID'])  # Только 1 аргумент!
                    
                    for drug in drugs:
                        chembl_id = drug["Drug ChEMBL ID"]
                        if chembl_id not in unique_drugs_for_gene or unique_drugs_for_gene[chembl_id]["pChEMBL value"] < drug["pChEMBL value"]:
                            unique_drugs_for_gene[chembl_id] = drug
                except Exception as e:
                    print(f"Ошибка при обработке информации о гене {gene_name}: {e}")
                    continue
            
            if not gene_info_list or 'UniProt_ID' not in gene_info_list[0]:
                continue
                
            for drug in unique_drugs_for_gene.values():
                result.append({
                    "Gene": gene_name,
                    "UniProt ID": gene_info_list[0]['UniProt_ID'],
                    "Protein name": gene_info_list[0]['Protein_Name'],
                    "Protein ChEMBL ID": gene_info_list[0]['ChEMBL_ID'],
                    **drug
                })
                
        except Exception as e:
            print(f"Ошибка при обработке гена {gene_name}: {e}")
            result.append({"Gene": gene_name, "Error": str(e)})
            continue

    return result

if __name__ == "__main__":
    # Замените этот путь на ваш реальный путь к файлу
    input_file = r"C:\Users\rusla\OneDrive\Рабочий стол\Диплом\Гены\thresholds\gene_selection\ALL\ALL_depscore_-0.15_zscore_-0.5.csv"
    
    # Чтение генов из файла
    try:
        genes_df = pd.read_csv(input_file)
        gene_names = genes_df['Gene'].tolist()
        print(f"Загружено {len(gene_names)} генов из файла")
    except Exception as e:
        print(f"Ошибка при чтении файла {input_file}: {e}")
        raise
    
    # Получаем имя выходного файла (без пути)
    output_filename = os.path.basename(input_file)
    
    start_time = time.time()
    print("\nНачинаем обработку с динамической фильтрацией pChEMBL")
    
    try:
        gene_drug_info = get_gene_drug_info(gene_names)
        
        # Определяем полный список всех возможных колонок
        all_columns = [
            "Gene", "UniProt ID", "Protein name", "Protein ChEMBL ID",
            "Drug ChEMBL ID", "Drug name", "Drug Mechanism ChEMBL",
            "Activity type ChEMBL", "Activity value ChEMBL",
            "pChEMBL value", "Max phase ChEMBL", "Action type ChEMBL", 
            "Mechanism of Action ChEMBL"
        ]
        
        # Создаем DataFrame из результатов
        df = pd.DataFrame(gene_drug_info)
        
        # Добавляем отсутствующие колонки и заполняем их NA
        for col in all_columns:
            if col not in df.columns:
                df[col] = pd.NA
        
        # Упорядочиваем колонки согласно нашему списку
        df = df[all_columns]
        
        # Фильтруем ошибочные записи (где нет Drug ChEMBL ID)
        df = df[df['Drug ChEMBL ID'].notna()]
        
        # Сохраняем в текущую директорию с тем же именем файла
        df.to_csv(output_filename, index=False, encoding='utf-8')
        
        elapsed_time = time.time() - start_time
        
        # Собираем статистику
        num_drugs = len(df)
        num_mechanisms = len(df[df['Drug Mechanism ChEMBL'] == 'Yes'])
        
        print(f"Файл {output_filename} сохранён. Время выполнения: {elapsed_time:.2f} секунд")
        print(f"Статистика: {num_drugs} препаратов, из них {num_mechanisms} с известным механизмом")
        
    except Exception as e:
        print(f"Критическая ошибка при обработке: {e}")

Загружено 1176 генов из файла

Начинаем обработку с динамической фильтрацией pChEMBL
Обрабатывается ген: ABHD11
Обрабатывается ген: ACAP3
Обрабатывается ген: ACIN1
Обрабатывается ген: ACLY
Обрабатывается ген: ACO2
Обрабатывается ген: ACPP
Обрабатывается ген: ACSL4
Обрабатывается ген: ACTL6A
Обрабатывается ген: ACTR10
Обрабатывается ген: ADAMTS4
Обрабатывается ген: ADGRB2
Обрабатывается ген: ADSL
Обрабатывается ген: ADSS2
Обрабатывается ген: AES
Обрабатывается ген: AFF1
Обрабатывается ген: AGPS
Обрабатывается ген: AGRN
Обрабатывается ген: AHCY
Обрабатывается ген: AHDC1
Обрабатывается ген: AJM1
Обрабатывается ген: AK2
Обрабатывается ген: AKAP17A
Обрабатывается ген: AKT2
Обрабатывается ген: ALAD
Обрабатывается ген: ALAS1
Обрабатывается ген: ALDH3B1
Обрабатывается ген: ALDH9A1
Обрабатывается ген: ALYREF
Обрабатывается ген: AMY1A&AMY1B&AMY1C
Обрабатывается ген: ANAPC7
Обрабатывается ген: ANKH
Обрабатывается ген: ANKRD34B
Обрабатывается ген: ANKRD36
Обрабатывается ген: ANKRD62P1-PARP4P3
Обра

In [15]:
import requests
from chembl_webresource_client.new_client import new_client
import pandas as pd
import numpy as np
from typing import List, Dict, Union, Optional
from functools import lru_cache
import time
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
import os
from pathlib import Path

# Настройка сессии с повторными попытками
session = requests.Session()
retries = Retry(
    total=5,
    backoff_factor=1,
    status_forcelist=[500, 502, 503, 504]
)
session.mount('https://', HTTPAdapter(max_retries=retries))

# Установка таймаута по умолчанию для ChEMBL клиента
new_client.timeout = 30

def uniprot_to_chembl(uniprot_id: str) -> Optional[str]:
    """Получает ChEMBL ID из UniProt ID."""
    target = new_client.target
    try:
        res = target.filter(target_components__accession=uniprot_id)
        res_df = pd.DataFrame(res)
        
        if res_df.empty:
            return None
        return res_df['target_chembl_id'].iloc[0]
    except Exception as e:
        print(f"Ошибка при получении ChEMBL ID для UniProt {uniprot_id}: {e}")
        return None

def search_gene_in_uniprot_filtered(gene_name: str, organism: str = "Homo sapiens") -> Union[str, List[Dict]]:
    """Ищет информацию о гене в UniProt."""
    url = "https://rest.uniprot.org/uniprotkb/search"
    query = f'gene_exact:"{gene_name}" AND reviewed:true AND taxonomy_name:"{organism}"'
    
    params = {
        "query": query,
        "format": "json",
        "fields": "accession,gene_names,protein_name",
        "size": 10
    }
    
    try:
        response = session.get(url, params=params, timeout=30)
        response.raise_for_status()
        
        results = response.json().get("results", [])
        if not results:
            return f"No results found for gene '{gene_name}'"
        
        gene_info = [
            {
                "UniProt_ID": result["primaryAccession"],
                "Gene_Names": result["genes"][0]["geneName"]["value"] if result.get("genes") else "N/A",
                "Protein_Name": result["proteinDescription"]["recommendedName"]["fullName"]["value"],
            }
            for result in results
        ]
        
        for gene in gene_info:
            uniprot_id = gene["UniProt_ID"]
            chembl_id = uniprot_to_chembl(uniprot_id)
            gene["ChEMBL_ID"] = chembl_id if chembl_id else f"No ChEMBL ID found for {uniprot_id}"
            time.sleep(0.1)  # Задержка между запросами
        
        return gene_info
    except Exception as e:
        print(f"Ошибка при поиске гена {gene_name}: {e}")
        return f"Error searching gene {gene_name}: {str(e)}"

@lru_cache(maxsize=1000)
def get_parent_info(chembl_id: str) -> dict:
    """Получает информацию о родительском соединении для данного ChEMBL ID."""
    molecule = new_client.molecule
    molecule_form = new_client.molecule_form
    
    try:
        # Проверяем, есть ли родительское соединение
        parent_data = list(molecule_form.filter(molecule_chembl_id=chembl_id).only('parent_chembl_id'))
        
        if parent_data and 'parent_chembl_id' in parent_data[0]:
            parent_id = parent_data[0]['parent_chembl_id']
        else:
            # Если нет родителя, считаем текущий ID родительским
            parent_id = chembl_id
        
        # Получаем информацию о родительском соединении
        mol_data = list(molecule.filter(molecule_chembl_id=parent_id).only('pref_name', 'molecule_chembl_id'))
        pref_name = mol_data[0].get('pref_name', 'Unknown') if mol_data else 'Unknown'
        
        return {
            'parent_chembl_id': parent_id,
            'parent_pref_name': pref_name,
            'is_parent': parent_id == chembl_id
        }
    except Exception as e:
        print(f"Error getting parent info for {chembl_id}: {e}")
        return {
            'parent_chembl_id': chembl_id,
            'parent_pref_name': 'Unknown',
            'is_parent': True
        }

def filter_parent_drugs(drugs_list: List[Dict]) -> List[Dict]:
    """Фильтрует список препаратов, оставляя по одному представителю из каждого семейства.
       Сохраняет 'Yes' в Drug Mechanism при замене дочернего с 'Yes' на родительский."""
    parent_groups = {}
    
    # 1. Группируем все препараты по семействам
    for drug in drugs_list:
        try:
            if not drug.get('Drug ChEMBL ID'):
                continue
                
            parent_info = get_parent_info(drug['Drug ChEMBL ID'])
            parent_id = parent_info['parent_chembl_id']
            
            if parent_id not in parent_groups:
                parent_groups[parent_id] = {
                    'parent_info': parent_info,
                    'drugs': [],
                    'has_mechanism': False
                }
            
            parent_groups[parent_id]['drugs'].append(drug)
            if drug.get('Drug Mechanism ChEMBL') == 'Yes':
                parent_groups[parent_id]['has_mechanism'] = True
                
        except Exception as e:
            print(f"Ошибка при группировке препаратов: {e}")
            continue
    
    filtered_drugs = []
    
    # 2. Обрабатываем каждое семейство
    for parent_id, group in parent_groups.items():
        drugs_in_group = group['drugs']
        has_mechanism = group['has_mechanism']
        
        # Если в группе только один препарат - оставляем как есть
        if len(drugs_in_group) == 1:
            filtered_drugs.append(drugs_in_group[0])
            continue
        
        # Находим родительский препарат в группе
        parent_drug = next(
            (d for d in drugs_in_group if d['Drug ChEMBL ID'] == parent_id),
            None
        )
        
        # Если есть родительский препарат
        if parent_drug:
            # Если в семействе есть препараты с механизмом, сохраняем 'Yes' для родителя
            if has_mechanism:
                parent_drug['Drug Mechanism ChEMBL'] = 'Yes'
            filtered_drugs.append(parent_drug)
        else:
            # Если нет родительского - выбираем препарат с максимальным pChEMBL
            best_drug = max(
                drugs_in_group,
                key=lambda x: float(x['pChEMBL value']) if x.get('pChEMBL value', 'N/A') != 'N/A' else -1
            )
            filtered_drugs.append(best_drug)
    
    return filtered_drugs

def get_action_type(chembl_id: str) -> str:
    try:
        client = new_client
        mechanisms = client.mechanism.filter(molecule_chembl_id=chembl_id)
        
        action_types = set()
        for mechanism in mechanisms:
            action_type = mechanism.get("action_type")
            if action_type:
                action_types.add(action_type)
        
        return "; ".join(action_types) if action_types else "N/A"
    except Exception as e:
        print(f"Ошибка при получении типа действия для {chembl_id}: {e}")
        return "N/A"

def get_mechanism_details(chembl_id: str) -> List[Dict]:
    try:
        client = new_client
        mechanisms = client.mechanism.filter(molecule_chembl_id=chembl_id)
        
        details = []
        for mechanism in mechanisms:
            mechanism_of_action = mechanism.get("mechanism_of_action", "N/A")
            
            details.append({
                "Mechanism of Action ChEMBL": mechanism_of_action,  # Изменено здесь
            })
        
        return details if details else [{"Mechanism of Action ChEMBL": "N/A"}]  # И здесь
    except Exception as e:
        print(f"Ошибка при получении механизма действия для {chembl_id}: {e}")
        return [{"Mechanism of Action ChEMBL": "N/A"}]  # И здесь

def get_max_phase(chembl_id: str) -> str:
    try:
        client = new_client
        molecule = client.molecule.get(chembl_id)
        max_phase = molecule.get("max_phase", None)
        
        if max_phase is None:
            return ""
        
        if max_phase == "0.0":
            return "Preclinical"
        elif max_phase == "1.0":
            return "Phase 1"
        elif max_phase == "2.0":
            return "Phase 2"
        elif max_phase == "3.0":
            return "Phase 3"
        elif max_phase == "4.0":
            return "Approved"
        else:
            return f"Unknown phase ({max_phase})"
    except Exception as e:
        print(f"Ошибка при получении фазы для {chembl_id}: {e}")
        return "N/A"

def get_pchembl_value(molecule_chembl_id: str, target_chembl_id: str) -> str:
    try:
        activities = new_client.activity.filter(
            molecule_chembl_id=molecule_chembl_id,
            target_chembl_id=target_chembl_id,
            pchembl_value__isnull=False
        ).only("pchembl_value")
        return activities[0]["pchembl_value"] if activities else "N/A"
    except Exception as e:
        print(f"Error fetching pChEMBL value for {molecule_chembl_id}: {e}")
        return "N/A"

def get_activity_data(molecule_chembl_id: str, target_chembl_id: str) -> Dict:
    """Возвращает объединенные данные активности: значение + единицы измерения"""
    try:
        activities = new_client.activity.filter(
            molecule_chembl_id=molecule_chembl_id,
            target_chembl_id=target_chembl_id,
            standard_type__in=["IC50", "Kd", "Ki", "EC50"]
        ).only("standard_value", "standard_type", "standard_units")
        
        if not activities:
            return {
                "Activity value ChEMBL": "N/A",
                "Activity type ChEMBL": "N/A"
            }
        
        activity = activities[0]
        value = activity.get('standard_value')
        units = activity.get('standard_units')
        act_type = activity.get('standard_type')
        
        if act_type == "Activity":
            act_type = "N/A"
        
        if value and units and value != "N/A" and units != "N/A":
            activity_value = f"= {value} {units}"
        else:
            activity_value = "N/A"
        
        return {
            "Activity value ChEMBL": activity_value,
            "Activity type ChEMBL": act_type
        }
    except Exception as e:
        print(f"Ошибка при получении данных активности для {molecule_chembl_id}: {e}")
        return {
            "Activity value ChEMBL": "N/A",
            "Activity type ChEMBL": "N/A"
        }

def get_drugs_for_target(chembl_id: str) -> List[Dict]:
    try:
        client = new_client
        
        mechanism_drugs = []
        mechanisms = client.mechanism.filter(target_chembl_id=chembl_id)
        for mech in mechanisms:
            try:
                drug_id = mech['molecule_chembl_id']
                drug = client.molecule.get(drug_id)
                
                activity_data = get_activity_data(drug_id, chembl_id)
                mechanism_details = get_mechanism_details(drug_id)
                
                if not mechanism_details:
                    print(f"Нет данных о механизме для {drug_id}")
                    continue
                    
                mechanism_drugs.append({
                    "Drug ChEMBL ID": drug_id,
                    "Drug name": drug.get('pref_name', 'N/A'),
                    "Drug Mechanism ChEMBL": "Yes",
                    "Activity value ChEMBL": activity_data["Activity value ChEMBL"],
                    "Activity type ChEMBL": activity_data["Activity type ChEMBL"],
                    "pChEMBL value": get_pchembl_value(drug_id, chembl_id),
                    "Max phase ChEMBL": get_max_phase(drug_id),
                    "Action type ChEMBL": get_action_type(drug_id),
                    "Mechanism of Action ChEMBL": mechanism_details[0]["Mechanism of Action ChEMBL"]
                })
                time.sleep(0.1)
            except Exception as e:
                print(f"Ошибка при обработке механизма {mech.get('molecule_chembl_id')}: {str(e)}")
                continue
        
        mechanism_ids = {drug["Drug ChEMBL ID"] for drug in mechanism_drugs}
        activity_drugs = []
        
        activities = client.activity.filter(
            target_chembl_id=chembl_id,
            pchembl_value__isnull=False,
            standard_type__in=["IC50", "Kd", "Ki", "EC50"]
        ).order_by('-pchembl_value')[:100]
        
        for activity in activities:
            try:
                if activity['molecule_chembl_id'] not in mechanism_ids:
                    value = activity.get('standard_value', 'N/A')
                    units = activity.get('standard_units', 'N/A')
                    act_type = activity.get('standard_type', 'N/A')
                    
                    if act_type == "Activity":
                        act_type = "N/A"
                    
                    activity_value = f"= {value} {units}" if value != "N/A" and units != "N/A" else "N/A"
                    
                    activity_drugs.append({
                        "Drug ChEMBL ID": activity['molecule_chembl_id'],
                        "Drug name": activity['molecule_pref_name'],
                        "Drug Mechanism ChEMBL": "No",
                        "Activity value ChEMBL": activity_value,  # Исправлено
                        "Activity type ChEMBL": act_type,          # Исправлено
                        "pChEMBL value": activity['pchembl_value'],
                        "Max phase ChEMBL": get_max_phase(activity['molecule_chembl_id']),  # Исправлено
                        "Action type ChEMBL": get_action_type(activity['molecule_chembl_id']),  # Исправлено
                        "Mechanism of Action ChEMBL": "N/A"
                    })
                    time.sleep(0.1)
            except Exception as e:
                print(f"Ошибка при обработке активности {activity}: {e}")
                continue
        
        all_drugs = mechanism_drugs + activity_drugs
        return filter_parent_drugs(all_drugs)
    except Exception as e:
        print(f"Ошибка при получении препаратов для цели {chembl_id}: {e}")
        return []

def get_gene_drug_info(gene_names: List[str]) -> List[Dict]:
    result = []
    
    for gene_name in gene_names:
        try:
            print(f"Обрабатывается ген: {gene_name}")
            gene_info_list = search_gene_in_uniprot_filtered(gene_name)
            
            if isinstance(gene_info_list, str):
                result.append({"Gene": gene_name, "Error": gene_info_list})
                continue
            
            unique_drugs_for_gene = {}

            for gene_info in gene_info_list:
                try:
                    if isinstance(gene_info['ChEMBL_ID'], str) and gene_info['ChEMBL_ID'].startswith("No ChEMBL"):
                        continue
                        
                    drugs = get_drugs_for_target(gene_info['ChEMBL_ID'])  # Только 1 аргумент!
                    
                    for drug in drugs:
                        chembl_id = drug["Drug ChEMBL ID"]
                        if chembl_id not in unique_drugs_for_gene or unique_drugs_for_gene[chembl_id]["pChEMBL value"] < drug["pChEMBL value"]:
                            unique_drugs_for_gene[chembl_id] = drug
                except Exception as e:
                    print(f"Ошибка при обработке информации о гене {gene_name}: {e}")
                    continue
            
            if not gene_info_list or 'UniProt_ID' not in gene_info_list[0]:
                continue
                
            for drug in unique_drugs_for_gene.values():
                result.append({
                    "Gene": gene_name,
                    "UniProt ID": gene_info_list[0]['UniProt_ID'],
                    "Protein name": gene_info_list[0]['Protein_Name'],
                    "Protein ChEMBL ID": gene_info_list[0]['ChEMBL_ID'],
                    **drug
                })
                
        except Exception as e:
            print(f"Ошибка при обработке гена {gene_name}: {e}")
            result.append({"Gene": gene_name, "Error": str(e)})
            continue

    return result

if __name__ == "__main__":
    # Замените этот путь на ваш реальный путь к файлу
    input_file = r"C:\Users\rusla\OneDrive\Рабочий стол\Диплом\Гены\thresholds\gene_selection\ALL\ALL_depscore_-0.15_zscore_-1.5.csv"
    
    # Чтение генов из файла
    try:
        genes_df = pd.read_csv(input_file)
        gene_names = genes_df['Gene'].tolist()
        print(f"Загружено {len(gene_names)} генов из файла")
    except Exception as e:
        print(f"Ошибка при чтении файла {input_file}: {e}")
        raise
    
    # Получаем имя выходного файла (без пути)
    output_filename = os.path.basename(input_file)
    
    start_time = time.time()
    print("\nНачинаем обработку с динамической фильтрацией pChEMBL")
    
    try:
        gene_drug_info = get_gene_drug_info(gene_names)
        
        # Определяем полный список всех возможных колонок
        all_columns = [
            "Gene", "UniProt ID", "Protein name", "Protein ChEMBL ID",
            "Drug ChEMBL ID", "Drug name", "Drug Mechanism ChEMBL",
            "Activity type ChEMBL", "Activity value ChEMBL",
            "pChEMBL value", "Max phase ChEMBL", "Action type ChEMBL", 
            "Mechanism of Action ChEMBL"
        ]
        
        # Создаем DataFrame из результатов
        df = pd.DataFrame(gene_drug_info)
        
        # Добавляем отсутствующие колонки и заполняем их NA
        for col in all_columns:
            if col not in df.columns:
                df[col] = pd.NA
        
        # Упорядочиваем колонки согласно нашему списку
        df = df[all_columns]
        
        # Фильтруем ошибочные записи (где нет Drug ChEMBL ID)
        df = df[df['Drug ChEMBL ID'].notna()]
        
        # Сохраняем в текущую директорию с тем же именем файла
        df.to_csv(output_filename, index=False, encoding='utf-8')
        
        elapsed_time = time.time() - start_time
        
        # Собираем статистику
        num_drugs = len(df)
        num_mechanisms = len(df[df['Drug Mechanism ChEMBL'] == 'Yes'])
        
        print(f"Файл {output_filename} сохранён. Время выполнения: {elapsed_time:.2f} секунд")
        print(f"Статистика: {num_drugs} препаратов, из них {num_mechanisms} с известным механизмом")
        
    except Exception as e:
        print(f"Критическая ошибка при обработке: {e}")

Загружено 49 генов из файла

Начинаем обработку с динамической фильтрацией pChEMBL
Обрабатывается ген: ARHGAP45
Обрабатывается ген: ATP1B3
Обрабатывается ген: ATP6V0A2
Обрабатывается ген: BCL2
Обрабатывается ген: C10orf95
Обрабатывается ген: C16orf52
Обрабатывается ген: C8orf86
Обрабатывается ген: CARS2
Обрабатывается ген: CBFB
Обрабатывается ген: CCND3
Обрабатывается ген: COQ4
Обрабатывается ген: DHODH
Обрабатывается ген: EBF1
Обрабатывается ген: ERG
Обрабатывается ген: FAM19A4
Обрабатывается ген: FAM214B
Обрабатывается ген: FAR1
Обрабатывается ген: FLJ42393
Обрабатывается ген: GCLC
Обрабатывается ген: IKZF1
Обрабатывается ген: LCE1F
Обрабатывается ген: LEF1
Обрабатывается ген: LOC107984065
Обрабатывается ген: LOC107987188
Обрабатывается ген: LOC440434
Обрабатывается ген: MAGEA3
Обрабатывается ген: MBNL1
Обрабатывается ген: MEF2C
Обрабатывается ген: MIR429
Обрабатывается ген: MON1A
Обрабатывается ген: MSL3P1
Обрабатывается ген: MYB
Обрабатывается ген: NMNAT1
Обрабатывается ген: PAICS


In [17]:
import requests
from chembl_webresource_client.new_client import new_client
import pandas as pd
import numpy as np
from typing import List, Dict, Union, Optional
from functools import lru_cache
import time
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
import os
from pathlib import Path

# Настройка сессии с повторными попытками
session = requests.Session()
retries = Retry(
    total=5,
    backoff_factor=1,
    status_forcelist=[500, 502, 503, 504]
)
session.mount('https://', HTTPAdapter(max_retries=retries))

# Установка таймаута по умолчанию для ChEMBL клиента
new_client.timeout = 30

def uniprot_to_chembl(uniprot_id: str) -> Optional[str]:
    """Получает ChEMBL ID из UniProt ID."""
    target = new_client.target
    try:
        res = target.filter(target_components__accession=uniprot_id)
        res_df = pd.DataFrame(res)
        
        if res_df.empty:
            return None
        return res_df['target_chembl_id'].iloc[0]
    except Exception as e:
        print(f"Ошибка при получении ChEMBL ID для UniProt {uniprot_id}: {e}")
        return None

def search_gene_in_uniprot_filtered(gene_name: str, organism: str = "Homo sapiens") -> Union[str, List[Dict]]:
    """Ищет информацию о гене в UniProt."""
    url = "https://rest.uniprot.org/uniprotkb/search"
    query = f'gene_exact:"{gene_name}" AND reviewed:true AND taxonomy_name:"{organism}"'
    
    params = {
        "query": query,
        "format": "json",
        "fields": "accession,gene_names,protein_name",
        "size": 10
    }
    
    try:
        response = session.get(url, params=params, timeout=30)
        response.raise_for_status()
        
        results = response.json().get("results", [])
        if not results:
            return f"No results found for gene '{gene_name}'"
        
        gene_info = [
            {
                "UniProt_ID": result["primaryAccession"],
                "Gene_Names": result["genes"][0]["geneName"]["value"] if result.get("genes") else "N/A",
                "Protein_Name": result["proteinDescription"]["recommendedName"]["fullName"]["value"],
            }
            for result in results
        ]
        
        for gene in gene_info:
            uniprot_id = gene["UniProt_ID"]
            chembl_id = uniprot_to_chembl(uniprot_id)
            gene["ChEMBL_ID"] = chembl_id if chembl_id else f"No ChEMBL ID found for {uniprot_id}"
            time.sleep(0.1)  # Задержка между запросами
        
        return gene_info
    except Exception as e:
        print(f"Ошибка при поиске гена {gene_name}: {e}")
        return f"Error searching gene {gene_name}: {str(e)}"

@lru_cache(maxsize=1000)
def get_parent_info(chembl_id: str) -> dict:
    """Получает информацию о родительском соединении для данного ChEMBL ID."""
    molecule = new_client.molecule
    molecule_form = new_client.molecule_form
    
    try:
        # Проверяем, есть ли родительское соединение
        parent_data = list(molecule_form.filter(molecule_chembl_id=chembl_id).only('parent_chembl_id'))
        
        if parent_data and 'parent_chembl_id' in parent_data[0]:
            parent_id = parent_data[0]['parent_chembl_id']
        else:
            # Если нет родителя, считаем текущий ID родительским
            parent_id = chembl_id
        
        # Получаем информацию о родительском соединении
        mol_data = list(molecule.filter(molecule_chembl_id=parent_id).only('pref_name', 'molecule_chembl_id'))
        pref_name = mol_data[0].get('pref_name', 'Unknown') if mol_data else 'Unknown'
        
        return {
            'parent_chembl_id': parent_id,
            'parent_pref_name': pref_name,
            'is_parent': parent_id == chembl_id
        }
    except Exception as e:
        print(f"Error getting parent info for {chembl_id}: {e}")
        return {
            'parent_chembl_id': chembl_id,
            'parent_pref_name': 'Unknown',
            'is_parent': True
        }

def filter_parent_drugs(drugs_list: List[Dict]) -> List[Dict]:
    """Фильтрует список препаратов, оставляя по одному представителю из каждого семейства.
       Сохраняет 'Yes' в Drug Mechanism при замене дочернего с 'Yes' на родительский."""
    parent_groups = {}
    
    # 1. Группируем все препараты по семействам
    for drug in drugs_list:
        try:
            if not drug.get('Drug ChEMBL ID'):
                continue
                
            parent_info = get_parent_info(drug['Drug ChEMBL ID'])
            parent_id = parent_info['parent_chembl_id']
            
            if parent_id not in parent_groups:
                parent_groups[parent_id] = {
                    'parent_info': parent_info,
                    'drugs': [],
                    'has_mechanism': False
                }
            
            parent_groups[parent_id]['drugs'].append(drug)
            if drug.get('Drug Mechanism ChEMBL') == 'Yes':
                parent_groups[parent_id]['has_mechanism'] = True
                
        except Exception as e:
            print(f"Ошибка при группировке препаратов: {e}")
            continue
    
    filtered_drugs = []
    
    # 2. Обрабатываем каждое семейство
    for parent_id, group in parent_groups.items():
        drugs_in_group = group['drugs']
        has_mechanism = group['has_mechanism']
        
        # Если в группе только один препарат - оставляем как есть
        if len(drugs_in_group) == 1:
            filtered_drugs.append(drugs_in_group[0])
            continue
        
        # Находим родительский препарат в группе
        parent_drug = next(
            (d for d in drugs_in_group if d['Drug ChEMBL ID'] == parent_id),
            None
        )
        
        # Если есть родительский препарат
        if parent_drug:
            # Если в семействе есть препараты с механизмом, сохраняем 'Yes' для родителя
            if has_mechanism:
                parent_drug['Drug Mechanism ChEMBL'] = 'Yes'
            filtered_drugs.append(parent_drug)
        else:
            # Если нет родительского - выбираем препарат с максимальным pChEMBL
            best_drug = max(
                drugs_in_group,
                key=lambda x: float(x['pChEMBL value']) if x.get('pChEMBL value', 'N/A') != 'N/A' else -1
            )
            filtered_drugs.append(best_drug)
    
    return filtered_drugs

def get_action_type(chembl_id: str) -> str:
    try:
        client = new_client
        mechanisms = client.mechanism.filter(molecule_chembl_id=chembl_id)
        
        action_types = set()
        for mechanism in mechanisms:
            action_type = mechanism.get("action_type")
            if action_type:
                action_types.add(action_type)
        
        return "; ".join(action_types) if action_types else "N/A"
    except Exception as e:
        print(f"Ошибка при получении типа действия для {chembl_id}: {e}")
        return "N/A"

def get_mechanism_details(chembl_id: str) -> List[Dict]:
    try:
        client = new_client
        mechanisms = client.mechanism.filter(molecule_chembl_id=chembl_id)
        
        details = []
        for mechanism in mechanisms:
            mechanism_of_action = mechanism.get("mechanism_of_action", "N/A")
            
            details.append({
                "Mechanism of Action ChEMBL": mechanism_of_action,  # Изменено здесь
            })
        
        return details if details else [{"Mechanism of Action ChEMBL": "N/A"}]  # И здесь
    except Exception as e:
        print(f"Ошибка при получении механизма действия для {chembl_id}: {e}")
        return [{"Mechanism of Action ChEMBL": "N/A"}]  # И здесь

def get_max_phase(chembl_id: str) -> str:
    try:
        client = new_client
        molecule = client.molecule.get(chembl_id)
        max_phase = molecule.get("max_phase", None)
        
        if max_phase is None:
            return ""
        
        if max_phase == "0.0":
            return "Preclinical"
        elif max_phase == "1.0":
            return "Phase 1"
        elif max_phase == "2.0":
            return "Phase 2"
        elif max_phase == "3.0":
            return "Phase 3"
        elif max_phase == "4.0":
            return "Approved"
        else:
            return f"Unknown phase ({max_phase})"
    except Exception as e:
        print(f"Ошибка при получении фазы для {chembl_id}: {e}")
        return "N/A"

def get_pchembl_value(molecule_chembl_id: str, target_chembl_id: str) -> str:
    try:
        activities = new_client.activity.filter(
            molecule_chembl_id=molecule_chembl_id,
            target_chembl_id=target_chembl_id,
            pchembl_value__isnull=False
        ).only("pchembl_value")
        return activities[0]["pchembl_value"] if activities else "N/A"
    except Exception as e:
        print(f"Error fetching pChEMBL value for {molecule_chembl_id}: {e}")
        return "N/A"

def get_activity_data(molecule_chembl_id: str, target_chembl_id: str) -> Dict:
    """Возвращает объединенные данные активности: значение + единицы измерения"""
    try:
        activities = new_client.activity.filter(
            molecule_chembl_id=molecule_chembl_id,
            target_chembl_id=target_chembl_id,
            standard_type__in=["IC50", "Kd", "Ki", "EC50"]
        ).only("standard_value", "standard_type", "standard_units")
        
        if not activities:
            return {
                "Activity value ChEMBL": "N/A",
                "Activity type ChEMBL": "N/A"
            }
        
        activity = activities[0]
        value = activity.get('standard_value')
        units = activity.get('standard_units')
        act_type = activity.get('standard_type')
        
        if act_type == "Activity":
            act_type = "N/A"
        
        if value and units and value != "N/A" and units != "N/A":
            activity_value = f"= {value} {units}"
        else:
            activity_value = "N/A"
        
        return {
            "Activity value ChEMBL": activity_value,
            "Activity type ChEMBL": act_type
        }
    except Exception as e:
        print(f"Ошибка при получении данных активности для {molecule_chembl_id}: {e}")
        return {
            "Activity value ChEMBL": "N/A",
            "Activity type ChEMBL": "N/A"
        }

def get_drugs_for_target(chembl_id: str) -> List[Dict]:
    try:
        client = new_client
        
        mechanism_drugs = []
        mechanisms = client.mechanism.filter(target_chembl_id=chembl_id)
        for mech in mechanisms:
            try:
                drug_id = mech['molecule_chembl_id']
                drug = client.molecule.get(drug_id)
                
                activity_data = get_activity_data(drug_id, chembl_id)
                mechanism_details = get_mechanism_details(drug_id)
                
                if not mechanism_details:
                    print(f"Нет данных о механизме для {drug_id}")
                    continue
                    
                mechanism_drugs.append({
                    "Drug ChEMBL ID": drug_id,
                    "Drug name": drug.get('pref_name', 'N/A'),
                    "Drug Mechanism ChEMBL": "Yes",
                    "Activity value ChEMBL": activity_data["Activity value ChEMBL"],
                    "Activity type ChEMBL": activity_data["Activity type ChEMBL"],
                    "pChEMBL value": get_pchembl_value(drug_id, chembl_id),
                    "Max phase ChEMBL": get_max_phase(drug_id),
                    "Action type ChEMBL": get_action_type(drug_id),
                    "Mechanism of Action ChEMBL": mechanism_details[0]["Mechanism of Action ChEMBL"]
                })
                time.sleep(0.1)
            except Exception as e:
                print(f"Ошибка при обработке механизма {mech.get('molecule_chembl_id')}: {str(e)}")
                continue
        
        mechanism_ids = {drug["Drug ChEMBL ID"] for drug in mechanism_drugs}
        activity_drugs = []
        
        activities = client.activity.filter(
            target_chembl_id=chembl_id,
            pchembl_value__isnull=False,
            standard_type__in=["IC50", "Kd", "Ki", "EC50"]
        ).order_by('-pchembl_value')[:100]
        
        for activity in activities:
            try:
                if activity['molecule_chembl_id'] not in mechanism_ids:
                    value = activity.get('standard_value', 'N/A')
                    units = activity.get('standard_units', 'N/A')
                    act_type = activity.get('standard_type', 'N/A')
                    
                    if act_type == "Activity":
                        act_type = "N/A"
                    
                    activity_value = f"= {value} {units}" if value != "N/A" and units != "N/A" else "N/A"
                    
                    activity_drugs.append({
                        "Drug ChEMBL ID": activity['molecule_chembl_id'],
                        "Drug name": activity['molecule_pref_name'],
                        "Drug Mechanism ChEMBL": "No",
                        "Activity value ChEMBL": activity_value,  # Исправлено
                        "Activity type ChEMBL": act_type,          # Исправлено
                        "pChEMBL value": activity['pchembl_value'],
                        "Max phase ChEMBL": get_max_phase(activity['molecule_chembl_id']),  # Исправлено
                        "Action type ChEMBL": get_action_type(activity['molecule_chembl_id']),  # Исправлено
                        "Mechanism of Action ChEMBL": "N/A"
                    })
                    time.sleep(0.1)
            except Exception as e:
                print(f"Ошибка при обработке активности {activity}: {e}")
                continue
        
        all_drugs = mechanism_drugs + activity_drugs
        return filter_parent_drugs(all_drugs)
    except Exception as e:
        print(f"Ошибка при получении препаратов для цели {chembl_id}: {e}")
        return []

def get_gene_drug_info(gene_names: List[str]) -> List[Dict]:
    result = []
    
    for gene_name in gene_names:
        try:
            print(f"Обрабатывается ген: {gene_name}")
            gene_info_list = search_gene_in_uniprot_filtered(gene_name)
            
            if isinstance(gene_info_list, str):
                result.append({"Gene": gene_name, "Error": gene_info_list})
                continue
            
            unique_drugs_for_gene = {}

            for gene_info in gene_info_list:
                try:
                    if isinstance(gene_info['ChEMBL_ID'], str) and gene_info['ChEMBL_ID'].startswith("No ChEMBL"):
                        continue
                        
                    drugs = get_drugs_for_target(gene_info['ChEMBL_ID'])  # Только 1 аргумент!
                    
                    for drug in drugs:
                        chembl_id = drug["Drug ChEMBL ID"]
                        if chembl_id not in unique_drugs_for_gene or unique_drugs_for_gene[chembl_id]["pChEMBL value"] < drug["pChEMBL value"]:
                            unique_drugs_for_gene[chembl_id] = drug
                except Exception as e:
                    print(f"Ошибка при обработке информации о гене {gene_name}: {e}")
                    continue
            
            if not gene_info_list or 'UniProt_ID' not in gene_info_list[0]:
                continue
                
            for drug in unique_drugs_for_gene.values():
                result.append({
                    "Gene": gene_name,
                    "UniProt ID": gene_info_list[0]['UniProt_ID'],
                    "Protein name": gene_info_list[0]['Protein_Name'],
                    "Protein ChEMBL ID": gene_info_list[0]['ChEMBL_ID'],
                    **drug
                })
                
        except Exception as e:
            print(f"Ошибка при обработке гена {gene_name}: {e}")
            result.append({"Gene": gene_name, "Error": str(e)})
            continue

    return result

if __name__ == "__main__":
    # Замените этот путь на ваш реальный путь к файлу
    input_file = r"C:\Users\rusla\OneDrive\Рабочий стол\Диплом\Гены\thresholds\gene_selection\ALL\ALL_depscore_-0.15_zscore_-1.csv"
    
    # Чтение генов из файла
    try:
        genes_df = pd.read_csv(input_file)
        gene_names = genes_df['Gene'].tolist()
        print(f"Загружено {len(gene_names)} генов из файла")
    except Exception as e:
        print(f"Ошибка при чтении файла {input_file}: {e}")
        raise
    
    # Получаем имя выходного файла (без пути)
    output_filename = os.path.basename(input_file)
    
    start_time = time.time()
    print("\nНачинаем обработку с динамической фильтрацией pChEMBL")
    
    try:
        gene_drug_info = get_gene_drug_info(gene_names)
        
        # Определяем полный список всех возможных колонок
        all_columns = [
            "Gene", "UniProt ID", "Protein name", "Protein ChEMBL ID",
            "Drug ChEMBL ID", "Drug name", "Drug Mechanism ChEMBL",
            "Activity type ChEMBL", "Activity value ChEMBL",
            "pChEMBL value", "Max phase ChEMBL", "Action type ChEMBL", 
            "Mechanism of Action ChEMBL"
        ]
        
        # Создаем DataFrame из результатов
        df = pd.DataFrame(gene_drug_info)
        
        # Добавляем отсутствующие колонки и заполняем их NA
        for col in all_columns:
            if col not in df.columns:
                df[col] = pd.NA
        
        # Упорядочиваем колонки согласно нашему списку
        df = df[all_columns]
        
        # Фильтруем ошибочные записи (где нет Drug ChEMBL ID)
        df = df[df['Drug ChEMBL ID'].notna()]
        
        # Сохраняем в текущую директорию с тем же именем файла
        df.to_csv(output_filename, index=False, encoding='utf-8')
        
        elapsed_time = time.time() - start_time
        
        # Собираем статистику
        num_drugs = len(df)
        num_mechanisms = len(df[df['Drug Mechanism ChEMBL'] == 'Yes'])
        
        print(f"Файл {output_filename} сохранён. Время выполнения: {elapsed_time:.2f} секунд")
        print(f"Статистика: {num_drugs} препаратов, из них {num_mechanisms} с известным механизмом")
        
    except Exception as e:
        print(f"Критическая ошибка при обработке: {e}")

Загружено 263 генов из файла

Начинаем обработку с динамической фильтрацией pChEMBL
Обрабатывается ген: ADSS2
Обрабатывается ген: AES
Обрабатывается ген: AFF1
Обрабатывается ген: AGPS
Обрабатывается ген: AHCY
Обрабатывается ген: AHDC1
Обрабатывается ген: AK2
Обрабатывается ген: ALAD
Обрабатывается ген: ALAS1
Обрабатывается ген: AMY1A&AMY1B&AMY1C
Обрабатывается ген: ANKH
Обрабатывается ген: ANKRD36
Обрабатывается ген: ARHGAP19-SLIT1
Обрабатывается ген: ARHGAP45
Обрабатывается ген: ATIC
Обрабатывается ген: ATP1B3
Обрабатывается ген: ATP6V0A2
Обрабатывается ген: AUH
Обрабатывается ген: BCL2
Обрабатывается ген: BCS1L
Обрабатывается ген: BIRC8
Обрабатывается ген: BRD9
Обрабатывается ген: C10orf95
Обрабатывается ген: C14orf2
Обрабатывается ген: C16orf52
Обрабатывается ген: C1QTNF5&MFRP
Обрабатывается ген: C1orf228
Обрабатывается ген: C2orf16
Обрабатывается ген: C8orf44-SGK3
Обрабатывается ген: C8orf86
Обрабатывается ген: CAB39
Обрабатывается ген: CAD
Обрабатывается ген: CARS2
Обрабатывается 